In [36]:
data = [
    #["CompanyName", "Callable", "Convertible", "Issue date", "Maturity date", "Coupon size", "Coupon frequency", "Coupon type", "Issue value", "Current value"],
    ["Nykredit", "NO", "NO", "21-07-2017", "21-07-2027", 0.05, 1, "Fixed", 100, 98],
    ["Nykredit", "NO", "NO", "21-07-2017", "21-07-2027", 0.035, 2, "Fixed", 100, 97]  
]

bonds = pd.DataFrame(data, columns=["Name", "Callable", "Convertible", "Issue date", "Maturity date", "Coupon size", "Coupon frequency", "Coupon type", "Issue value", "Current value"])
bonds

Name Callable Convertible  Issue date Maturity date  Coupon size  \
0  Nykredit       NO          NO  21-07-2017    21-07-2027        0.050   
1  Nykredit       NO          NO  21-07-2017    21-07-2027        0.035   

   Coupon frequency Coupon type  Issue value  Current value  
0                 1       Fixed          100             98  
1                 2       Fixed          100             97

In [33]:
d0 = date(int(bonds['Issue date'][0][-4:12]), int(bonds['Issue date'][0][3:5]), int(bonds['Issue date'][0][0:2]))
dx = date.today()
d1 = date(int(bonds['Maturity date'][0][-4:12]), int(bonds['Maturity date'][0][3:5]), int(bonds['Maturity date'][0][0:2]))
delta = d1 - d0
print("Years from issue to maturity:",round(delta.days/365,1))
delta2 = d1 - dx
print("Years from date to maturity:",round(delta2.days/365,1))

Years from issue to maturity: 10.0
Years from date to maturity: 4.0


In [43]:
def BP(data):
    coupon = data['Coupon size'] * data['Coupon frequency']
    dx = date.today()
    d1 = date(int(data['Maturity date'][-4:12]), int(data['Maturity date'][3:5]), int(data['Maturity date'][0:2]))
    delta = d1 = dx
    maturity = round(delta2.days/365,1)
    r = 0.05
    facevalue = data['Issue value']
    
    return coupon*(1-(1+r)**(-maturity))/r + facevalue/((1+r)**maturity)


In [47]:
BP(bonds.iloc[0])

82.4475450043963

In [ ]:
#1. find loss distribution
#2. calculate risk measures, (EL e.i. mean) Expected loss, (UEL e.i. var) unexpected loss, expected shortfall

In [171]:
a = independentBinomialLossDistribution(1,1000,0.1,100)
b = independentPoissonLossDistribution(1,1000,0.1,100)

In [124]:
alpha = list([0.85,0.925,0.95,0.975])
computeRiskMeasures(1000, a, alpha)

(9.3,
 29.043243620504928,
 array([  0., 100., 100., 100.]),
 array([ 62., 100., 100., 100.]))

In [169]:
aa = independentBinomialAnalytic(100,0.05,100,alpha)
print(aa)
ba = independentPoissonAnalytic(100,100,0.5,alpha)
print(ba)

(array([5.92052922e-003, 3.11606801e-002, 8.11817719e-002, 1.39575678e-001,
       1.78142642e-001, 1.80017827e-001, 1.50014856e-001, 1.06025537e-001,
       6.48708880e-002, 3.49012965e-002, 1.67158841e-002, 7.19822760e-003,
       2.80983446e-003, 1.00107462e-003, 3.27419144e-004, 9.88001628e-005,
       2.76250455e-005, 7.18422237e-006, 1.74353935e-006, 3.96039408e-007,
       8.44189265e-008, 1.69261005e-008, 3.19895202e-009, 5.70979994e-010,
       9.64154814e-011, 1.54264770e-011, 2.34207647e-012, 3.37843390e-013,
       4.63582095e-014, 6.05769707e-015, 7.54555249e-016, 8.96754964e-017,
       1.01769889e-017, 1.10372447e-018, 1.14472972e-019, 1.13612273e-020,
       1.07964879e-021, 9.82895059e-023, 8.57650813e-024, 7.17602570e-025,
       5.75970484e-026, 4.43622966e-027, 3.27991917e-028, 2.32846159e-029,
       1.58758745e-030, 1.03982336e-031, 6.54351083e-033, 3.95688225e-034,
       2.29950394e-035, 1.28436310e-036, 6.89500191e-038, 3.55779252e-039,
       1.76449224e-040, 

In [170]:
####################################################
#               LOSS DISTRIBUTIONS                 #
####################################################

def independentBinomialLossDistribution(N,M,p,c, fullOutput = 0):
    """
    N = number of counterparties
    M = number of simulations
    p = probability of default
    c = exposure
    """
    U = np.random.uniform(0,1,[M,N])
    defaultIndicator = 1*np.less(U,p)
    lossDistribution = np.sort(np.dot(defaultIndicator,c),axis=None)
    if fullOutput==0:
        return lossDistribution
    else:
        return lossDistribution,defaultIndicator

def independentPoissonLossDistribution(N,M,p,c, fullOutput = 0):
    """
    N = number of counterparties
    M = number of simulations
    p = probability of default
    c = exposure
    """
    lam = -np.log(1-p)
    H = np.random.poisson(lam,[M,N])
    defaultIndicator = 1*np.greater_equal(H,1)
    lossDistribution = np.sort(np.dot(defaultIndicator,c),axis=None)
    if fullOutput==0:
        return lossDistribution
    else:
        return lossDistribution,defaultIndicator

####################################################
#               RISK MEASURES                      #
####################################################   

def expectedLoss(lossDistribution):
    expectedLoss = np.mean(lossDistribution)
    return expectedLoss

def unExpectedLoss(lossDistribution):
    unExpectedLoss = np.std(lossDistribution)
    return unExpectedLoss
    
def computeRiskMeasures(M,lossDistribution,alpha):
    
    #if alpha is integer than what?
    """
    M = iterations
    alpha = confidence level
    EL = expected loss, how much do we expect to loss given the loss dist.
    UEL = unexpected loss, how much could we end up losing, given the loss dist.
    ES = how much could we at max expect to loss? given loss dist.
    var = VaR value at risk
    """  
    EL = np.mean(lossDistribution)
    UEL = np.std(lossDistribution)
    ES = np.zeros([len(alpha)])
    var = np.zeros([len(alpha)])
    for j in range(0,len(alpha)):
        ES[j] = np.mean(lossDistribution[np.ceil(alpha[j]*(M-1)).astype(int): M])        
        var[j] = lossDistribution[np.ceil(alpha[j]*(M-1)).astype(int)]
    return EL, UEL, var, ES     

####################################################
#               PMFs, CDFs, var, es                #
####################################################  

def independentBinomialAnalytic(N,p,c,alpha):
    """
    N = number of counter parties
    p = probability of default
    c = exposure?
    alpha = confidence level
    """
    pmfBinomial = np.zeros(N+1)
    for k in range(0,N+1):
        pmfBinomial[k] =  getBC(N,k)*(p**k)*((1-p)**(N-k))
    cdfBinomial = np.cumsum(pmfBinomial)
    varAnalytic = c*np.interp(alpha,cdfBinomial,np.linspace(0,N,N+1))
    esAnalytic = analyticExpectedShortfall(N,alpha,pmfBinomial,c)
    return pmfBinomial,cdfBinomial,varAnalytic,esAnalytic

def independentPoissonAnalytic(N,c,myLam,alpha):
    """
    N = number of counter parties
    p = probability of default
    c = exposure?
    alpha = confidence level
    """
    pmfPoisson = np.zeros(N+1)
    for k in range(0,N+1):
        pmfPoisson[k] =  poissonDensity(myLam,k)
    cdfPoisson = np.cumsum(pmfPoisson)
    varAnalytic = c*np.interp(alpha,cdfPoisson,np.linspace(0,N,N+1))
    esAnalytic = analyticExpectedShortfall(N,alpha,pmfPoisson,c)
    return pmfPoisson,cdfPoisson,varAnalytic,esAnalytic 

####################################################
#               LOSS DISTRIBUTIONS                 #
####################################################

def analyticExpectedShortfall(N,alpha,pmf,c):
    """
    N = number of counter parties
    alpha = confidence level
    c = exposure?
    pmf = probability mass function
    """
    cdf = np.cumsum(pmf)
    numberDefaults = np.linspace(0,N,N+1)
    expectedShortfall = np.zeros(len(alpha))
    for n in range(0,len(alpha)):   
        myAlpha = np.linspace(alpha[n],1,1000)
        nanCheck =  ~np.isnan(pmf)
        loss = c*np.interp(myAlpha,cdf[nanCheck],numberDefaults[nanCheck])
        prob = np.interp(loss,numberDefaults[nanCheck],pmf[nanCheck])
        expectedShortfall[n] = np.dot(loss,prob)/np.sum(prob)
    return expectedShortfall
    
####################################################
#               LOSS DISTRIBUTIONS                 #
####################################################

def independentBinomialSimulation(N,M,p,c,alpha):
    """
    N = number of counterparties
    M = number of simulations
    p = probability of default
    c = exposure
    alpha = confidence level
    """
    lossDistribution = independentBinomialLossDistribution(N,M,p,c,alpha)
    el,ul,var,es = computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es

def independentPoissonSimulation(N,M,p,c,alpha):
    """
    N = number of counterparties
    M = number of simulations
    p = probability of default
    c = exposure
    alpha = 
    """
    lossDistribution = independentPoissonLossDistribution(N,M,p,c,alpha)
    el,ul,var,es = computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es

####################################################




In [ ]:
# Let's instantiate bonds that is defined in the Bond_Utils.py module

rhea = BU.Bond(100, 0.75, 1, 2, "Rhea", -1)
europa = BU.Bond(100, 1.50, 1, 2, "Europa", -1)
ganymede = BU.Bond(100, 2.50, 1, 2, "Ganymede", -1)


# Yield volatilities
dy_rhea = 3.40
dy_europa = 3.00
dy_ganymede = 2.00


# Rhea YTM
coupon_rhea = 0.0075/2 * 100
fv_rhea = 100
price_rhea = 98.5

ytm_rhea = Symbol('y')
equation = (coupon_rhea*exp(-ytm_rhea*0.5)+
            coupon_rhea*exp(-ytm_rhea*1.0)+
            fv_rhea*exp(-ytm_rhea*1.0)-price_rhea)

y_rhea= solve(equation, ytm_rhea)
y_rhea[0]


# Europa YTM
coupon_europa = 0.015/2 * 100
fv_europa = 100
price_europa = 99

ytm_europa = Symbol('y')
equation = (coupon_europa*exp(-ytm_europa*0.5)+
            coupon_europa*exp(-ytm_europa*1)+
            coupon_europa*exp(-ytm_europa*1.5)+
            coupon_europa*exp(-ytm_europa*2)+
            coupon_europa*exp(-ytm_europa*2.5)+
            fv_europa*exp(-ytm_europa*2.5)-price_europa)

y_europa= solve(equation, ytm_europa)
y_europa[0]

# Ganymede YTM
coupon_ganymede = 0.025/2 * 100
fv_ganymede = 100
price_ganymede = 100.5

ytm_ganymede = Symbol('y')
equation = (coupon_ganymede*exp(-ytm_ganymede*0.5)+
            coupon_ganymede*exp(-ytm_ganymede*1)+
            coupon_ganymede*exp(-ytm_ganymede*1.5)+
            coupon_ganymede*exp(-ytm_ganymede*2)+
            coupon_ganymede*exp(-ytm_ganymede*2.5)+
            coupon_ganymede*exp(-ytm_ganymede*3.0)+
            coupon_ganymede*exp(-ytm_ganymede*3.5)+
            coupon_ganymede*exp(-ytm_ganymede*4.0)+
            coupon_ganymede*exp(-ytm_ganymede*4.5)+
            coupon_ganymede*exp(-ytm_ganymede*5.0)+
            fv_ganymede*exp(-ytm_ganymede*5.0)-price_ganymede)

y_ganymede= solve(equation, ytm_ganymede)
y_ganymede[0]



# MD for Rhea
md_rhea = rhea.duration(float(y_rhea[0]), 'modified')
md_rhea


# MD for Europa
md_europa = europa.duration(float(y_europa[0]), 'modified')
md_europa

# MD for Ganymede
md_ganymede = ganymede.duration(float(y_ganymede[0]), 'modified')
md_europa


# Bond return volatility for Rhea
md_rhea*dy_rhea


# Bond return volatility for Europa
md_europa*dy_europa


# Bond return volatility for Ganymede
md_ganymede*dy_ganymede


# Note: we have calculated YTM for Ganymede bond in b)
y_ganymede[0]

credit_spread = 0.01
ytm_cb = y_ganymede[0] + credit_spread
ytm_cb


fv_cb = 100000
coupon_cb = 0.03/2 * fv_cb
price_cb = coupon_cb*math.exp(-ytm_cb*0.5) +\
           coupon_cb*math.exp(-ytm_cb*1.0) +\
           coupon_cb*math.exp(-ytm_cb*1.5) +\
           coupon_cb*math.exp(-ytm_cb*2.0) +\
           coupon_cb*math.exp(-ytm_cb*2.5) +\
           coupon_cb*math.exp(-ytm_cb*3.0) +\
           coupon_cb*math.exp(-ytm_cb*3.5) +\
           coupon_cb*math.exp(-ytm_cb*4.0) +\
           coupon_cb*math.exp(-ytm_cb*4.5) +\
           coupon_cb*math.exp(-ytm_cb*5.0) +\
           fv_cb*math.exp(-ytm_cb*5.0)

price_cb

#fixed income derivatives

spot_rates = np.array([3.005, 3.575, 3.925, 4.134, 4.412, 4.499, 4.785, 4.896, 5.001, 5.069])/100
maturity_years = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

swap_rate = 0.05
face_value = 1000000
years_to_maturity = 5 


fixedLegValue = [face_value * swap_rate for i in range(years_to_maturity * 2)]
fixedLegValue[-1] += face_value
fixedLegCashFlows = [fv * np.exp(-rate * year) for fv, year, rate in zip(fixedLegValue, maturity_years, spot_rates)]
fixedLegValue = sum(fixedLegCashFlows)

print("Value of the fixed leg is {:.2f}$".format(fixedLegValue))


flaotingLegValue = [face_value * rate for rate in spot_rates]
flaotingLegValue[-1] = face_value
flaotingLegCashFlows = [fv * np.exp(-rate * year) for fv, year, rate in zip(flaotingLegValue, maturity_years, spot_rates)]
flaotingLegValue = sum(flaotingLegCashFlows)

print("Value of the floating leg is {:.2f}$".format(flaotingLegValue))

bondPrice = fixedLegValue - flaotingLegValue
print("Bond price is {:.2f}$".format(bondPrice))

X, i = sp.symbols('X i')
spot_rates = Array(spot_rates)
maturity_years = Array(maturity_years)

eq = sp.summation((1_000_000*X)**(-spot_rates[i]*maturity_years[i]), (i, 0, 9)) + (1_000_000*(1+X))**(-spot_rates[-1]*maturity_years[-1])

s = solve(eq - 1114757, X)

print("In order for the price to be 0, the swap rate should be {:f}%".format(s[0]*100))
#s = 0.0383715


spot_rates = np.array([3.005, 3.575, 3.925, 4.134, 4.412, 4.499, 4.785, 4.896, 5.001, 5.069])/100
maturity_years = np.array([0.5, 1, 1.5, 2, 2.5, 3, 3.5, 4, 4.5, 5])

calculated_swap_rate = 0.0383715
face_value = 1000000
years_to_maturity = 5

floatingLegValue = 1154099

fixedLegValue = [face_value * calculated_swap_rate for i in range(years_to_maturity * 2)]
fixedLegValue[-1] += face_value
fixedLegCashFlows = [fv * np.exp(-rate * year) for fv, year, rate in zip(fixedLegValue, maturity_years, spot_rates)]
fixedLegValue = sum(fixedLegCashFlows)

bondPrice = fixedLegValue - flaotingLegValue

print(bondPrice)

In [110]:
def jointDefaultProbability(p,q,myRho):
    pr,err=nInt.quad(jointIntegrand,-5,5,args=(p,q,myRho))
    return pr

def jointIntegrand(g,p,q,myRho):
    p1 = th.computeP(p,myRho,g)
    p2 = th.computeP(q,myRho,g)
    f = p1*p2*util.gaussianDensity(g,0,1)
    return f

def initializeRegion2r(N,rStart):
    startRegion = np.zeros(N)
    w = np.cumsum(rStart)
    u = np.random.uniform(0,1,N)
    for n in range(0,N):
        if ((u[n]>0) & (u[n]<=w[0])):
            startRegion[n] = 0
        elif ((u[n]>w[0]) & (u[n]<1)):
            startRegion[n] = 1
    return startRegion    

def createRatingData(K,N,T,Pin,wStart,myRho,nu,isT): 
    Q = cumulateTransitionMatrix(K,Pin)
    if isT==1:
        Delta = transformCumulativeTransitionMatrix_t(K,Q,nu)                            
    else:
        Delta = transformCumulativeTransitionMatrix(K,Q)                    
    Y = np.zeros([N,T]) # latent variables
    X = np.zeros([N,T]) # credit states  
    allP = np.zeros([N,T]) # default probabilities
    Xlast = mc.initializeCounterparties(N,wStart) # initial states
    X0 = Xlast
    Plast = Pin[(Xlast-1).astype(int),-1] 
    for t in range(0,T):
        Y[:,t] = th.getY(N,1,Plast,myRho,nu,isT)    
        for n in range(0,N):
            if Xlast[n] == 4:
                X[n,t] = 4
                continue
            else:
                X[n,t] = migrateRating(Xlast[n],Delta,Y[n,t])
        allP[:,t] = Pin[(Xlast-1).astype(int),-1]
        Plast = allP[:,t]
        Xlast = X[:,t]
    return X,Y,Delta,allP,X0    
    
def simulateCorrelatedTransitionData(K,N,T,Pin,wStart,myRho): 
    Q = cumulateTransitionMatrix(K,Pin)
    Delta = transformCumulativeTransitionMatrix(K,Q)                    
    Y = np.zeros([N,T]) # latent variables
    X = np.zeros([N,T]) # credit states  
    allP = np.zeros([N,T]) # default probabilities
    Xlast = mc.initializeCounterparties(N,wStart) # initial states
    X0 = Xlast
    Plast = Pin[(Xlast-1).astype(int),-1] 
    for t in range(0,T):
        Y[:,t] = th.getY(N,1,Plast,myRho)    
        for n in range(0,N):
            if Xlast[n] == 4:
                X[n,t] = 4
                continue
            else:
                X[n,t] = migrateRating(Xlast[n],Delta,Y[n,t])
        allP[:,t] = Pin[(Xlast-1).astype(int),-1]
        Plast = allP[:,t]
        Xlast = X[:,t]
    return X,Y,Delta,allP,X0    


def createRatingData2r(K,N,T,P,wStart,rStart,myRho,nu,isT): 
    Q = cumulateTransitionMatrix(K,P)
    Delta = transformCumulativeTransitionMatrix(K,Q)
    rId = initializeRegion2r(N,rStart).astype(int)
    Y = np.zeros([N,T]) # latent variables
    X = np.zeros([N,T]) # credit states
    allP = np.zeros([N,T]) # default probabilities
    Xlast = mc.initializeCounterparties(N,wStart) # initial states
    X0 = Xlast
    Plast = P[(Xlast-1).astype(int),-1] 
    for t in range(0,T):
        Y[:,t] = th.getY2r(N,1,Plast,myRho,rId,nu,P,isT)    
        for n in range(0,N):
            if Xlast[n] == 4:
                X[n,t] = 4
                continue
            else:
                X[n,t] = migrateRating(Xlast[n],Delta,Y[n,t])
        allP[:,t] = P[(Xlast-1).astype(int),-1]
        Plast = allP[:,t]
        Xlast = X[:,t]
    return X,Y,Delta,allP,X0,rId           
    
def migrateRating(lastX,Delta,myY):
        transitionRow = (lastX-1).astype(int)
        myMap = Delta[transitionRow,:]    
        if myY>=myMap[1]:
            myX = 1
        elif (myY<myMap[1]) & (myY>=myMap[2]):
            myX = 2
        elif (myY<myMap[2]) & (myY>=myMap[3]):
            myX = 3
        elif myY<myMap[3]:
            myX = 4
        return myX

def cumulateTransitionMatrix(K,M):
    H =  np.zeros([K,K]) 
    for n in range(0,K):
        for m in range(0,K):
            H[m,(K-1)-n] = np.sum(M[m,(K-1)-n:K])
    return H

def transformCumulativeTransitionMatrix(K,M_c):    
    H = np.zeros([K,K])
    for n in range(0,K):
        for m in range(0,K):
            if M_c[n,m]>=0.9999999:  
                H[n,m]=5
            elif M_c[n,m]<=0.0000001:
                H[n,m] = -5
            else:
                H[n,m] = norm.ppf(M_c[n,m])
    return H    

def transformCumulativeTransitionMatrix_t(K,M_c,nu):    
    # Element-by-element inverse-normal transform 
    # of the cumulative transition matrix
    H = np.zeros([K,K])
    for n in range(0,K):
        for m in range(0,K):
            if M_c[n,m]>=0.9999999:  
                H[n,m] = 5
            elif M_c[n,m]<=0.0000001:
                H[n,m] = -5
            else:
                H[n,m] = myT.ppf(M_c[n,m],nu)
    return H

def getSimpleEstimationData(T,X,allP):
    N,T = X.shape
    kVec = np.zeros(T)
    nVec = np.zeros(T)
    pVec = np.zeros(T)
    kVec[0] = np.sum(X[:,0]==4)
    nVec[0] = N
    pVec[0] = np.mean(allP[:,0])
    for t in range(1,T):
        kVec[t] = np.sum(X[:,t]==4)-np.sum(X[:,t-1]==4)
        nVec[t] = nVec[t-1] - kVec[t-1]
        pVec[t] = np.mean(allP[X[:,t-1]!=4,t])
    return pVec,nVec,kVec  

def get2rEstimationData(T,X,X0,rId,allP,numP):
    N,T = X.shape
    kMat = np.zeros([T,numP])
    nMat = np.zeros([T,numP])
    pMat = np.zeros([T,numP])
    for m in range(0,numP):
        xLoc = (rId==m).astype(bool)
        kMat[0,m] = np.sum(X[xLoc,0]==4)
        nMat[0,m] = np.sum(xLoc)
        pMat[0,m] = np.mean(allP[xLoc,0])
        for t in range(1,T):
            kMat[t,m] = np.sum(X[xLoc,t]==4)-np.sum(X[xLoc,t-1]==4)
            nMat[t,m] = nMat[t-1,m] - kMat[t-1,m]
            if np.sum(xLoc)==0:
                pMat[t,m] = 0.0                
            else:
                pMat[t,m] = np.mean(allP[(X[xLoc,t-1]!=4).astype(int),t])
    return pMat,nMat,kMat     
    
def getCMF(g,myRho,myP,myN,myK):
    pg = th.computeP(myP,myRho,g)
    f=util.getBC(myN,myK)*np.power(pg,myK)*np.power(1-pg,myN-myK)
    cmf = f*util.gaussianDensity(g,0,1)        
    return cmf

def logLSimple(x,T,pVec,nVec,kVec):
    L = np.zeros(T)
    for t in range(0,T):
        L[t],err = nInt.quad(getCMF,-5,5,
                     args=(x,pVec[t],nVec[t],kVec[t]))
    logL = np.sum(np.log(L))
    return -logL          

def maxSimpleLogL(T,pVec,nVec,kVec):
    myBounds = ((0.001,0.999),)                           
    xStart = 0.5
    r = scipy.optimize.minimize(logLSimple,
                    xStart,args=(T,pVec,nVec,kVec), 
                    method='TNC',jac=None,bounds=myBounds,
                    options={'maxiter':1000}) 
    return r.x,r.success    

def computeSimpleScore(x0,T,pVec,nVec,kVec):
    h = 0.00001
    fUp = logLSimple(x0+h/2,T,pVec,nVec,kVec)
    fDown = logLSimple(x0-h/2,T,pVec,nVec,kVec)    
    score = np.divide(fUp-fDown,h)
    return score

def simpleFisherInformation(x0,T,pVec,nVec,kVec):
    h = 0.000001
    f = logLSimple(x0,T,pVec,nVec,kVec)    
    fUp = logLSimple(x0+h,T,pVec,nVec,kVec)
    fDown = logLSimple(x0-h,T,pVec,nVec,kVec)    
    I = -np.divide(fUp-2*f+fDown,h**2)
    return I

def getProdCMF(g,myRho,myP,myN,myK):
    pg = th.computeP(myP,myRho,g)
    return np.multiply(util.getBC(myN,myK),np.power(pg,myK)*np.power(1-pg,myN-myK))

def getCMF2r(g,myRho,pVec3,nVec3,kVec3):
    myF=getProdCMF(g,myRho,pVec3,nVec3,kVec3)
    return np.prod(myF)*util.gaussianDensity(g,0,1)
   
def logL2r(x,T,pMat,nMat,kMat):
    L = np.zeros(T)
    for t in range(0,T):
        L[t],err = nInt.quad(getCMF2r,-5,5,args=(x,pMat[t,:],nMat[t,:],kMat[t,:]))
    return -np.sum(np.log(L))

def log2rGridSearch(gridSize,numVar,T,pMat,nMat,kMat):
    rhoRange = np.linspace(0.001,0.999,gridSize)
    bigF = np.zeros([gridSize,gridSize])
    rhoGrid = np.zeros([gridSize**numVar,numVar])
    startTime = time.time()
    for n in range(0,gridSize):
        for m in range(0,gridSize):
            print("Running: n:%d, m: %d" % (n+1,m+1))
            rhoGrid = np.array([rhoRange[n],rhoRange[m]])
            bigF[n,m] = -logL2r(rhoGrid,T,pMat,nMat,kMat)
    print("Loop takes %d minutes." % ((time.time()-startTime)/60))
    # Find the values associated with the biggest value of OF
    bigMax = np.max(bigF)
    for n in range(0,gridSize):
        for m in range(0,gridSize):
                if (bigF[n,m]==bigMax):
                    myN = n
                    myM = m
    rhoStart = np.array([rhoRange[myN],rhoRange[myM]])
    return rhoStart,bigF,rhoRange  

def max2rLogL(T,pMat,nMat,kMat,xStart):
    myBounds = ((0.001,0.999),(0.001,0.999))                           
    r = scipy.optimize.minimize(logL2r,
                    xStart,args=(T,pMat,nMat,kMat), 
                    method='TNC',jac=None,bounds=myBounds,
                    options={'maxiter':100})    
    return r.x,r.success    
   
def hessian2r(x0,epsilon,T,pMat,nMat,kMat):
    # The first derivative
    f1 = approx_fprime(x0,logL2r,epsilon,T,pMat,nMat,kMat) 
    n = x0.shape[0]
    hessian = np.zeros([n,n])
    xx = x0
    for j in range(0,n):
        xx0 = xx[j] # Store old value
        xx[j] = xx0 + epsilon # Perturb with finite difference
        # Recalculate the partial derivatives for this new point
        f2 = approx_fprime(xx, logL2r,epsilon,T,pMat,nMat,kMat) 
        hessian[:, j] = (f2 - f1)/epsilon # scale...
        xx[j] = xx0 # Restore initial value of x0        
    return hessian    

def score2r(x0,epsilon,T,pMat,nMat,kMat):
    score = approx_fprime(x0,logL2r,epsilon,T,pMat,nMat,kMat) 
    return score    

def mapRating(D,from_value,to_value,K):
    if (to_value==K) & (from_value!=K):
        d_u = D[from_value-1,to_value-1]
        d_l = -5
    elif (to_value==K) & (from_value==K):
        d_u = -5
        d_l = -5
    else:
        d_u = D[from_value-1,to_value]
        d_l = D[from_value-1,to_value-1]
    return d_l, d_u

def mapRatingData(Y,D,K):
    N,T = Y.shape
    d_low = np.zeros([N,T-1])
    d_upp = np.zeros([N,T-1])
    for n in range(0,N):
        for m in range(1,T):
            d_low[n,m-1],d_upp[n,m-1] = mapRating(D,
                                 Y[n,m-1].astype(int),
                                 Y[n,m].astype(int),K)
    return d_low,d_upp

def buildCorrelationMatrix1F(x,N):
    R = x*np.ones([N,N])+np.eye(N)
    R[R==1+x] = 1    
    return R

def logLCopula(x,X,nu):
    M,T = X.shape
    R = buildCorrelationMatrix1F(x,M)
    detR = anp.det(R)
    Rinv = anp.inv(R)
    V = 0
    for t in range(0,T):
        V += np.log(1+np.divide(np.dot(np.dot(X[:,t],Rinv),X[:,t]),nu))
    return -(-0.5*(T*np.log(detR)+(nu+M)*V))

def scoreCopula(x0,myZ,nu):
    h = 0.000000001
    fUp = -logLCopula(x0+h/2,myZ,nu)
    fDown = -logLCopula(x0-h/2,myZ,nu)    
    score = np.divide(fUp-fDown,h)
    return score

def hessianCopula(x0,myZ,nu):
    h = 0.0001
    f = logLCopula(x0,myZ,nu)    
    fUp = logLCopula(x0+h,myZ,nu)
    fDown = logLCopula(x0-h,myZ,nu)    
    I = np.divide(fUp-2*f+fDown,h**2)
    return I

def maxCopulaLogL(startX,myZ,nu):
    myBounds = ((0.001,0.999),)                           
    r = scipy.optimize.minimize(logLCopula,
                        startX,args=(myZ,nu), 
                        method='TNC',jac=None,bounds=myBounds,
                        options={'maxiter':1000}) 
    return r.x,r.success    
    

def mixtureMethodOfMoment(x,myP,myV,myModel):
    if myModel==0: # Beta-binomial
        M1 = mix.betaMoment(x[0],x[1],1)
        M2 = mix.betaMoment(x[0],x[1],2)
    elif myModel==1: # Logit
        M1,err = nInt.quad(mix.logitProbitMoment,-8,8,args=(x[0],x[1],1,1)) 
        M2,err = nInt.quad(mix.logitProbitMoment,-8,8,args=(x[0],x[1],2,1))
    elif myModel==2: # Probit
        M1,err = nInt.quad(mix.logitProbitMoment,-8,8,args=(x[0],x[1],1,0)) 
        M2,err = nInt.quad(mix.logitProbitMoment,-8,8,args=(x[0],x[1],2,0))
    elif myModel==3: # Poisson-gamma
        M1 = mix.poissonGammaMoment(x[0],x[1],1)
        M2 = mix.poissonGammaMoment(x[0],x[1],2)
    elif myModel==4: # Poisson-lognormal
        M1,err = nInt.quad(mix.poissonMixtureMoment,0.0001,0.9999,args=(x[0],x[1],1,0)) 
        M2,err = nInt.quad(mix.poissonMixtureMoment,0.0001,0.9999,args=(x[0],x[1],2,0)) 
    elif myModel==5: # Poisson-Weibull
        M1,err = nInt.quad(mix.poissonMixtureMoment,0.0001,0.9999,args=(x[0],x[1],1,1)) 
        M2,err = nInt.quad(mix.poissonMixtureMoment,0.0001,0.9999,args=(x[0],x[1],2,1)) 
    f1 = M1 - myP    
    f2 =(M2-M1**2) - myV    
    return [1e4*f1, 1e4*f2]
        
def integrateGaussianMoment(g,r,myP,myMoment):
    integrand = np.power(th.computeP(myP,r,g),myMoment)
    return integrand*util.gaussianDensity(g,0,1)

def methodOfMomentsG(x,myP,myV):
    if (x<=0) | (x>1):
        return [100,100]
    M1,err = nInt.quad(integrateGaussianMoment,-5,5,args=(x[0],myP,1)) 
    M2,err = nInt.quad(integrateGaussianMoment,-5,5,args=(x[0],myP,2)) 
    f1 = (M2-M1**2) - myV
    return 1e4*f1

def thresholdMoment(g,w,p1,p2,myP,whichModel,myMoment,invCdf=0):
    d1 = util.gaussianDensity(g,0,1)
    if whichModel==1: # t
        d2 = util.chi2Density(w,p2)
        integrand = np.power(th.computeP_t(myP,p1,g,w,p2),myMoment)
    if whichModel==2: # Variance-gamma
        d2 = util.gammaDensity(w,p2,p2)
        integrand = np.power(th.computeP_NVM(myP,p1,g,w,p2,invCdf),myMoment)
    if whichModel==3: # Generalized hyperbolic
        d2 = util.gigDensity(w,p2)
        integrand = np.power(th.computeP_NVM(myP,p1,g,w,p2,invCdf),myMoment)
    return integrand*d1*d2

def getThresholdMoments(x,myP,whichModel):
    if whichModel==0: # Gaussian
        M1,err = nInt.quad(integrateGaussianMoment,-5,5,args=(x[0],myP,1)) 
        M2,err = nInt.quad(integrateGaussianMoment,-5,5,args=(x[0],myP,2)) 
    elif whichModel==1: # t
        lowerBound = np.maximum(x[1]-40,2)
        support = [[-7,7],[lowerBound,x[1]+40]]
        M1,err=nInt.nquad(thresholdMoment,support,args=(x[0],x[1],myP,whichModel,1))
        M2,err=nInt.nquad(thresholdMoment,support,args=(x[0],x[1],myP,whichModel,2))
    elif whichModel==2: # Variance-gamma
        invCdf = th.nvmPpf(myP,x[1],0)
        support = [[-7,7],[0,100]]
        M1,err=nInt.nquad(thresholdMoment,support,args=(x[0],x[1],myP,whichModel,1,invCdf))
        M2,err=nInt.nquad(thresholdMoment,support,args=(x[0],x[1],myP,whichModel,2,invCdf))
    elif whichModel==3: # Generalized hyperbolic        
        invCdf = th.nvmPpf(myP,x[1],1)
        support = [[-7,7],[0,100]]
        M1,err=nInt.nquad(thresholdMoment,support,args=(x[0],x[1],myP,whichModel,1,invCdf))
        M2,err=nInt.nquad(thresholdMoment,support,args=(x[0],x[1],myP,whichModel,2,invCdf))
    return M1,M2

def thresholdMethodOfMoment(x,myP,myV,whichModel):
    if (x[0]<=0) | (x[0]>1):
        return 100
    M1,M2 = getThresholdMoments(x,myP,whichModel)
    f1 = M1 - myP    
    f2 =(M2-M1**2) - myV    
    return [1e4*f1,1e4*f2]

def getThresholdDefaultCorrelation(x,myP,whichModel):
    if whichModel==0: # Gaussian
        jp = th.jointDefaultProbability(myP,myP,x[0])
    elif whichModel==1: # t
        jp = th.jointDefaultProbabilityT(myP,myP,x[0],x[1])
    elif whichModel==2: # Variance-gamma
        jp = th.jointDefaultProbabilityNVM(myP,myP,x[0],x[1],0)
    elif whichModel==3: # Generalized hyperbolic
        jp = th.jointDefaultProbabilityNVM(myP,myP,x[0],x[1],1)
    return np.divide(jp-myP**2,myP*(1-myP))

def getCMF_cr(s,myW,myA,myP,myN,myK):
    ps=myP*(1-myW)+myP*myW*s
    f=util.getBC(myN,myK)*np.power(ps,myK)*np.power(1-ps,myN-myK)
    return f*util.gammaDensity(s,myA,myA)        

def logLSimple_cr(x,T,pVec,nVec,kVec,myA):
    L = np.zeros(T)
    for t in range(0,T):
        L[t],err = nInt.quad(getCMF_cr,0,100,
                     args=(x,myA,pVec[t],nVec[t],kVec[t]))
    return -np.sum(np.log(L))

def crPlusMoment(s,myW,myA,myP,momentNumber):
    v0 = myP*(1-myW) + myP*myW*s
    myDensity = util.gammaDensity(s,myA,myA)
    return np.power(v0,momentNumber)*myDensity

def crPlusScore(x0,T,pVec,nVec,kVec,myA):
    h = 0.0000001
    fUp = logLSimple_cr(x0+h/2,T,pVec,nVec,kVec,myA)
    fDown = logLSimple_cr(x0-h/2,T,pVec,nVec,kVec,myA)    
    return np.divide(fUp-fDown,h)

def crPlusFisherInformation(x0,T,pVec,nVec,kVec,myA):
    h = 0.000001
    f = logLSimple_cr(x0,T,pVec,nVec,kVec,myA)    
    fUp = logLSimple_cr(x0+h,T,pVec,nVec,kVec,myA)
    fDown = logLSimple_cr(x0-h,T,pVec,nVec,kVec,myA)    
    return -np.divide(fUp-2*f+fDown,h**2)

def jointDefaultProbabilityRegion(p,q,rhoVec):
    pr,err=nInt.quad(jointIntegrandRegion,-10,10,args=(p,q,rhoVec))
    return pr

def jointIntegrandRegion(g,p,q,rhoVec):
    p1 = th.computeP(p,rhoVec[0],g)
    p2 = th.computeP(q,rhoVec[1],g)
    density = util.gaussianDensity(g,0,1)
    f = p1*p2*density
    return f

def getRegionalDefaultCorrelation(rhoVec,myP):
    jp = jointDefaultProbabilityRegion(myP[0],myP[1],rhoVec)
    return np.divide(jp-myP[0]*myP[1],np.sqrt(myP[0]*(1-myP[0]))*np.sqrt(myP[1]*(1-myP[1])))

In [119]:


class myPrinter(object):
    def __init__(self, *files):
        self.files = files
    def write(self, obj):
        for f in self.files:
            f.write(obj)
            f.flush() 
    def flush(self) :
        for f in self.files:
            f.flush()    

def printLine(R,myString):
    M = len(R)
    for m in range(0,M):
        if m!=(M-1):
            print(myString % (R[m]) + " & ", end=" ")
        else:
            print(myString % (R[m]) + "\\\\", end="\n")
               
def printMatrix(R,myString):
    N,M = R.shape
    for n in range(0,N):
        for m in range(0,M):
            if m!=(M-1):
                print(myString % (R[n,m]) + " & ", end=" ")
            else:
                print(myString % (R[n,m]) + "\\\\", end="\n")            
  

def simulateDefaultProbabilities(N,pMean):
    p = (pMean/1.5)*np.random.chisquare(1.5,N)
    return p

def simulateTenors(N,T):
    tenor = np.random.uniform(0.25,T,N)
    return tenor
    
def simulateRegions(N,rStart):
    w = np.cumsum(rStart)
    u = np.random.uniform(0,1,N)
    myRegion = np.zeros(N)
    for n in range(0,N):
        if ((u[n]>=0) & (u[n]<=w[0])):
            myRegion[n] = 1
        elif ((u[n]>w[0]) & (u[n]<=w[1])):
            myRegion[n] = 2
        elif (u[n]>w[1]) & (u[n]<=1):
            myRegion[n] = 3        
    return myRegion    
    
def simulateExposures(N,portfolioSize):
    w = np.random.weibull(1,N)
    c = portfolioSize*np.divide(w,np.sum(w)) 
    return c
        
def generateGamma(a,b,N):
    G1 = np.random.gamma(a,1,N)
    G2 = np.random.gamma(b,1,N)
    Z = np.divide(G1,G1+G2)  
    return Z
    
def computeBeta(a,b):
    Ga = math.gamma(a)
    Gb = math.gamma(b)
    Gab = math.gamma(a+b)
    return (Ga*Gb)/Gab
   

In [ ]:
importlib.reload(util)
importlib.reload(bp)
importlib.reload(mix)
importlib.reload(th)
importlib.reload(mert)

def getQ(p_n,myH=-50):
    return np.divide(1-np.exp(myH*p_n),1-np.exp(myH))

def getRho(p_n,rhoMin=0.12,rhoMax=0.24):
    myQ = getQ(p_n)
    return rhoMin*myQ + rhoMax*(1-myQ)
    
def getMaturitySlope(p_n,p0=0.11582,p1=-0.05478):
    return np.power(p0+p1*np.log(p_n),2)
                                  
def getMaturityAdjustment(tenor,p_n,p0=2.5):
    myB = getMaturitySlope(p_n)
    return np.divide(1+(tenor-p0)*myB,1-(p0-1)*myB)    
                              
def getBaselK(p_n,tenor,alpha):
    g = norm.ppf(1-alpha)
    rhoBasel = getRho(p_n)
    ma = getMaturityAdjustment(tenor,p_n)
    pG = th.computeP(p_n,rhoBasel,g)
    return np.multiply(pG-p_n,ma)
    
def getBaselRiskCapital(p_n,tenor,c,myAlpha):
    myCounter = len(myAlpha)    
    riskCapital = np.zeros(myCounter)
    for n in range(0,myCounter):
        riskCapitalCoefficient=getBaselK(p_n,tenor,myAlpha[n])
        riskCapital[n] = np.dot(c,riskCapitalCoefficient)
    return riskCapital     
    
def runModelSuite(N,M,P,C,alpha,nu,myRho,rhoTarget,tenor,modelType):
    startTime = time.time() 
    if modelType==0: # Binomial (independent-default) model
        el,ul,var,es = bp.independentBinomialSimulation(N,M,P,C,alpha)
        simTime = (time.time() - startTime)
    elif modelType==1: # Gaussian threshold model
        el,ul,var,es = th.oneFactorThresholdModel(N,M,P,C,myRho,nu,alpha,0)
        simTime = (time.time() - startTime)
    elif modelType==2: # Beta-binomial mixture model
        myP = np.mean(P)
        a,b = mix.betaCalibrate(myP,rhoTarget)
        M1 = mix.betaMoment(a,b,1)
        M2 = mix.betaMoment(a,b,2)
        print("a, b parameters are %0.1f and %0.1f." % (a,b))
        print("Targeted: %0.4f and calibrated: %0.4f default probability." % (myP,M1))
        print("Targeted: %0.3f and calibrated: %0.3f default correlation." % (rhoTarget,np.divide(M2-M1**2,M1-M1**2)))
        el,ul,var,es = mix.betaBinomialSimulation(N,M,C,a,b,alpha)
        simTime = (time.time() - startTime)
    elif modelType==3: # t-distributed threshold model
        el,ul,var,es = th.oneFactorThresholdModel(N,M,P,C,myRho,nu,alpha,1)
        simTime = (time.time() - startTime)
    elif modelType==4: # Basel IRB approach
        mAdjustedC = np.multiply(C,getMaturityAdjustment(tenor,P))
        el = np.dot(P,mAdjustedC)       
        var = getBaselRiskCapital(P,tenor,C,alpha)
        ul = np.sum(P*(1-P)*mAdjustedC)
        es = var
        simTime = (time.time() - startTime)    
    elif modelType==5: # Asymptotic Gaussian threshold model (ASRF)
        meanP = np.maximum(0.0009,np.median(P))
        a,b = th.getAsrfMoments(meanP,myRho)
        el = np.sum(C)*a
        ul = np.sum(C)*b
        pdf,cdf,var,es = th.asrfModel(meanP,myRho,C,alpha)
        simTime = (time.time() - startTime)    
    return el,ul,var,es,simTime
    
def fG(myAlpha):
    return norm.pdf(norm.ppf(1-myAlpha))
    
def dfG(myAlpha):
    z = norm.ppf(1-myAlpha)
    return -z*fG(myAlpha)
  
def mu(myAlpha,myP,myC,myRho):
    pn = th.computeP(myP,myRho,norm.ppf(1-myAlpha))
    return np.dot(myC,pn)

def dmu(myAlpha,myP,myC,myRho):
    constant = np.sqrt(np.divide(myRho,1-myRho))
    ratio = norm.ppf(th.computeP(myP,myRho,norm.ppf(1-myAlpha)))
    return -constant*np.dot(myC,norm.pdf(ratio))

def d2mu(myAlpha,myP,myC,myRho):
    constant = np.divide(myRho,1-myRho)
    ratio = norm.ppf(th.computeP(myP,myRho,norm.ppf(1-myAlpha)))
    return -constant*np.dot(ratio*myC,norm.pdf(ratio))

def nu(myAlpha,myP,myC,myRho):
    pn = th.computeP(myP,myRho,norm.ppf(1-myAlpha))
    return np.dot(np.power(myC,2),pn*(1-pn))

def dnu(myAlpha,myP,myC,myRho):
    pn = th.computeP(myP,myRho,norm.ppf(1-myAlpha))
    ratio = norm.ppf(pn)
    constant = np.sqrt(np.divide(myRho,1-myRho))
    return -constant*np.dot(norm.pdf(ratio)*np.power(myC,2),1-2*pn)

def granularityAdjustment(myAlpha,myP,myC,myRho):
    # Get the necessary functions and their derivatives
    f = fG(myAlpha)    
    df = dfG(myAlpha)            
    dg = dmu(myAlpha,myP,myC,myRho) 
    dg2 = d2mu(myAlpha,myP,myC,myRho) 
    h = nu(myAlpha,myP,myC,myRho)    
    dh = dnu(myAlpha,myP,myC,myRho)  
    # Build and return granularity adjustment formula
    t1 = np.reciprocal(dg)
    t2 = np.divide(h*df,f)+dh
    t3 = np.divide(h*dg2,np.power(dg,2))    
    return -0.5*(t1*t2-t3)
  
def getW(myP,myA,myRho,myAlpha):
    num = th.computeP(myP,myRho,norm.ppf(1-myAlpha))-myP
    den = myP*(gamma.ppf(myAlpha,myA,0,1/myA)-1)
    return np.divide(num,den)
  
def getC(gBar,xi):
    gVar = xi*gBar*(1-gBar)
    return np.divide(gBar**2+gVar,gBar)

def getRK(gBar,myA,myW,myP,myAlpha):
    q = gamma.ppf(myAlpha,myA,0,1/myA)
    return gBar*myP*(1-myW+myW*q)
  
def getK(gBar,myA,myW,myP,myAlpha):
    q = gamma.ppf(myAlpha,myA,0,1/myA)
    return gBar*myP*myW*(q-1)

def getDelta(myA,myAlpha):
    q = gamma.ppf(myAlpha,myA,0,1/myA)
    return (q-1)*(myA+np.divide(1-myA,q))

def myLGDRatio(gBar,xi):
    gVar = xi*gBar*(1-gBar)
    return np.divide(gVar,gBar**2)

def granularityAdjustmentCR(myA,myW,gBar,xi,p,c,myAlpha,isApprox=0):
    myDelta = getDelta(myA,myAlpha)
    Cn = getC(gBar,xi)
    RKn = getRK(gBar,myA,myW,p,myAlpha)
    Kn = getK(gBar,myA,myW,p,myAlpha)
    KStar = np.dot(c,Kn)
    myRatio = myLGDRatio(gBar,xi)
    if isApprox==0:
        t1 = myDelta*(Cn*RKn+np.power(RKn,2)*myRatio)
        t2 = Kn*(Cn+2*RKn*myRatio)
    else:
        t1 = myDelta*Cn*RKn
        t2 = Kn*Cn  
    return np.dot(np.power(c,2),t1-t2)/(2*KStar)  
  
def gaContribution(myA,myW,gBar,xi,p,c,n,myAlpha,isApprox=0):
    myDelta = getDelta(myA,myAlpha)
    Cn = getC(gBar,xi)
    RKn = getRK(gBar,myA,myW,p,myAlpha)
    Kn = getK(gBar,myA,myW,p,myAlpha)
    KStar = np.sum(c*Kn)
    myRatio = myLGDRatio(gBar,xi)
    if isApprox==0:
        t1 = myDelta*(Cn*RKn+np.power(RKn,2)*myRatio)
        t2 = Kn*(Cn+2*RKn*myRatio)
    else:
        t1 = myDelta*Cn*RKn
        t2 = Kn*Cn  
    return np.dot(np.power(c[n],2),t1[n]-t2[n])/(2*KStar)      

In [ ]:


importlib.reload(util)

def inventedTransitionMatrix():
    P = np.array([[0.960,0.029,0.010,0.001],
              [0.100,0.775,0.120,0.005],
              [0.120,0.220,0.650,0.010],
              [0.000,0.000,0.000,1.00]])
    return P

def initializeCounterparties(N,wStart):
    startRating = np.zeros(N)
    w = np.cumsum(wStart)
    u = np.random.uniform(0,1,N)
    for n in range(0,N):
        if ((u[n]>0) & (u[n]<=w[0])):
            startRating[n] = 1
        elif ((u[n]>w[0]) & (u[n]<=w[1])):
            startRating[n] = 2
        elif ((u[n]>w[1]) & (u[n]<=w[2])):
            startRating[n] = 3        
        elif u[n]>w[2]:
            startRating[n] = 4
    return startRating

def simulateRatingData(N,T,P,wStart):
    tStart = initializeCounterparties(N,wStart)
    D = np.zeros([N,T])
    for n in range(0,N):
        D[n,0] = tStart[n]
        for t in range(1,T):
            D[n,t] = transitionStep(D[n,t-1].astype(int),P)
    return D
    
def transitionStep(currentState,P):
    myP = np.cumsum(P[currentState-1,:])
    u = np.random.uniform(0,1)
    if ((u>0) & (u<=myP[0])):
        return 1
    elif ((u>myP[0]) & (u<=myP[1])):
        return 2
    elif ((u>myP[1]) & (u<=myP[2])):
        return 3        
    elif ((u>myP[2]) & (u<=myP[3])):
        return 4    
    
def getTransitionCount(K,T,N,data):
    n_ij = np.zeros([K,K])
    # Count the n_ij or n_{to,from} 
    # => the number of j's followed by i's.
    for i in range(0,K): # to
        for j in range(0,K): # from
            for n in range(0,N): # obligor
                for tau in range(1,T): # time
                    if ((data[n,tau]==j+1) & (data[n,tau-1]==i+1)):
                        #if data[n,tau]==K:
                            # Absorbing-state (no exit)
                        n_ij[i,j] += 1
                            #break
                        #else:
                        #    n_ij[i,j] += 1   
    #if np.sum(n_ij[-1,0])==0:
    #    n_ij[-1,-1]=1
    return n_ij    

def estimateCohortTransitionMatrix(K,n_ij,myPeriod):
    H = np.zeros([K,K])
    for i in range(0,K):
        for j in range(0,K):
            H[i,j] = np.divide(n_ij[i,j],np.sum(n_ij[i,:]))
    H[-1,:] = np.zeros(K)
    H[-1,-1] = 1        
    return anp.matrix_power(H,myPeriod)

def estimateHazardRateTransitionMatrix(K,n_ij,D,myPeriod):
    # Uses the hazard-rate technique
    H = np.zeros([K,K])
    # Construct the generator matrix
    for i in range(0,K):
        for j in range(0,K):
            if i==j:
                continue
            H[i,j] = np.divide(n_ij[i,j],np.sum(D==i+1))
    for i in range(0,K):
        H[i,i]=-(np.sum(H[i,:]-H[i,i]))
    M = asp.expm(myPeriod*H)
    M[-1,:] = np.zeros(K)
    M[-1,-1] = 1   
    return M
   
def bootstrapDistribution(K,N,T,PEstimate,wStart,S):
    PBootstrap = np.zeros([K,K,S])
    for s in range(0,S):
        if np.remainder(s+1,500)==0:
            print("Run iteration %d" % (s+1))
        DBootstrap = simulateRatingData(N,T,PEstimate,wStart)
        NBootstrap = getTransitionCount(K,T,N,DBootstrap)
        PBootstrap[:,:,s] = estimateCohortTransitionMatrix(K,NBootstrap,1)
    return PBootstrap
    

def estimateTransitionMatrix(K,T,N,data,myPeriod,whichModel):
    n_ij = getTransitionCount(K,T,N,data)
    if whichModel==0:
        M = estimateCohortTransitionMatrix(K,n_ij,myPeriod)
    elif whichModel==1:
        M = estimateHazardRateTransitionMatrix(K,n_ij,data,myPeriod)
    return M    

def tpLikelihood(x,M): 
    mVector = np.reshape(M, len(M)**2)
    L = 0
    for i in range(0,len(x)):
        if x[i]<=0:
            pass
        else:
            L += mVector[i]*np.log(x[i])
    return L   

def hessian(x0,epsilon,M):
    # The first derivative
    f1 = approx_fprime(x0,tpLikelihood,epsilon,M) 
    n = x0.shape[0]
    hessian = np.zeros([n,n])
    xx = x0
    for j in range(0,n):
        xx0 = xx[j] # Store old value
        xx[j] = xx0 + epsilon # Perturb with finite difference
        # Recalculate the partial derivatives for this new point
        f2 = approx_fprime(xx, tpLikelihood,epsilon,M) 
        hessian[:, j] = (f2 - f1)/epsilon # scale...
        xx[j] = xx0 # Restore initial value of x0        
    return hessian  
    
    
def printSEConfidenceInterval(myP,se,T):
    K = myP.shape[0]
    coeff = myT.ppf(1-0.05/2,T-1)
    for i in range(0,K):
        for j in range(0,K):
            low = np.maximum(myP[i,j]-coeff*se[i,j],0)
            up = np.minimum(myP[i,j]+coeff*se[i,j],1)
            if j!=(K-1):
                print("[%0.2f, %0.2f]" % (low,up) + " & ", end=" ")
            else:
                print("[%0.2f, %0.2f]" % (low,up) + "\\\\", end="\n")

def printQuantileConfidenceInterval(myP,S):
    K = myP.shape[0]
    for i in range(0,K):
        for j in range(0,K):
            sortP = np.sort(myP[i,j,:],axis=None)
            low = sortP[np.ceil(0.025*(S-1)).astype(int)]
            up = sortP[np.ceil(0.975*(S-1)).astype(int)]
            if j!=(K-1):
                print("[%0.2f, %0.2f]" % (low,up) + " & ", end=" ")
            else:
                print("[%0.2f, %0.2f]" % (low,up) + "\\\\", end="\n")

def bLikelihood(N,k,pDomain):
    L = util.getBC(N,k)*(pDomain**k)*((1-pDomain)**(N-k))
    return L/np.abs(np.max(L))

def bLogLikelihood(N,k,pDomain):
    ell = k*np.log(pDomain) + (N-k)*np.log(1-pDomain)
    return ell/np.abs(np.max(ell))
    
def bScore(N,k,pDomain):
    s = np.divide(k,pDomain)-np.divide(N-k,1-pDomain)
    return s
    
def bFisherInformation(N,k,pDomain):
    a = np.divide(k,np.power(pDomain,2))
    b = np.divide(N-k,np.power(1-pDomain,2))
    I = a + b 
    return I                
  
def getNSSpotCurve(l,s,c,t,v=0.10):
    den = v*t
    level = 1
    slope = np.divide(1-np.exp(-v*t),den)
    curve = np.divide(1-np.exp(-v*t),den) - np.exp(-v*t)
    return l*level + s*slope + c*curve

def getStep(myT,tenor,h):
    myRange = np.insert(tenor,0,0)
    for n in range(0,len(tenor)):
        if myT==0:
            loc = 0
        elif (myT>myRange[n]) & (myT<=myRange[n+1]):
            loc = n
        elif myT>myRange[n+1]:
            loc = -1
    try: return h[loc]
    except:
        print("No value found!")
 
def survivalProb(myT,tenor,h):    
    myH = getStep(myT,tenor,h)    
    return np.exp(-myH*myT)

def defaultDensity(myT,tenor,h):
    myH = getStep(myT,tenor,h)    
    myS = survivalProb(myT,tenor,h)
    return myH*myS

def getCouponStream(beta,fRate,tenor,h,Delta,d):    
    myCoupon = 0
    for i in range(0,beta):
        myCoupon += fRate*Delta*np.interp((i+1)*Delta,tenor,d)* \
                survivalProb((i+1)*Delta,tenor,h)
    return myCoupon

def getPremiumAccrual(beta,fRate,tenor,h,Delta,d):
    myAccrual = 0
    for i in range(0,beta):
        sIncrement = survivalProb(i*Delta,tenor,h) - survivalProb((i+1)*Delta,tenor,h)
        myAccrual += fRate*(Delta/2)*np.interp(i*Delta+(Delta/2),tenor,d)*sIncrement
    return myAccrual

def getProtectionPayment(beta,fRate,tenor,h,Delta,d,R):
    myProtection = 0
    for i in range(0,beta):
        sIncrement = survivalProb(i*Delta,tenor,h) - survivalProb((i+1)*Delta,tenor,h)        
        myProtection += (1-R)*np.interp(i*Delta+(Delta/2),tenor,d)*sIncrement
    return myProtection

def cdsPrice(h,cds,tenor,Delta,d,R):
    K = len(h)
    cStream = np.zeros(K)
    aStream = np.zeros(K)
    xStream = np.zeros(K)
    beta = np.zeros(K)
    for n in range(0,K):
        beta = (tenor[n]/Delta).astype(int)
        cStream[n] = getCouponStream(beta,cds[n],tenor,h,Delta,d)
        aStream[n] = getPremiumAccrual(beta,cds[n],tenor,h,Delta,d)
        xStream[n] = getProtectionPayment(beta,cds[n],tenor,h,Delta,d,R)
    return cStream+aStream-xStream    


In [ ]:
import numpy as np
import math
import cmUtilities as util
import numpy.linalg as anp
import importlib 
from scipy.stats import norm
import scipy.integrate as nInt
from scipy.stats import t as myT
from scipy.stats import mvn
from scipy.optimize import minimize

import thresholdModels as th


def generateCorrelationMatrix(N,rho):
    lowerDiagonal = np.zeros([N,N])
    for n in range(0,N):
        for m in range(0,N):
            if n==m:
                lowerDiagonal[n][m] = 0.5
            elif n>m:
                lowerDiagonal[n][m] = rho
            else:
                continue                
    C = np.transpose(lowerDiagonal)+lowerDiagonal
    return C
        
def getK(mu,sigma,dt,A,myP):
    t1 = (mu-0.5*(np.power(sigma,2)))*dt
    t2 = np.multiply(np.multiply(norm.ppf(myP),sigma),np.sqrt(dt))
    return np.multiply(A,np.exp(t1+t2))   

def mertonIndirectSimulation(N,M,p,Omega,c,alpha):
    Z = np.random.normal(0,1,[M,N])
    w,v = anp.eigh(Omega)
    H = np.dot(v,np.sqrt(np.diag(w)))
    xi = np.dot(Z,np.transpose(H)) 
    K = norm.ppf(p)*np.ones((M,1))        
    lossIndicator = 1*np.less(xi,K)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)    
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es             

def mertonDirectSimulation(N,M,K,hatA,Omega,c,alpha):
    Z = np.random.normal(0,1,[M,N])
    w,v = anp.eigh(Omega)
    H = np.dot(v,np.sqrt(np.diag(w)))
    A = np.tile(hatA,(M,1)) + np.dot(Z,np.transpose(H)) 
    lossIndicator = 1*np.less(A,K)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)    
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es         

def assignRating(rateList):
    rv = np.random.uniform()
    if rv>0 and rv <= 0.25: # AAA
        myRating = rateList[0]         
    elif rv>0.25 and rv <= 0.50: # AA
        myRating = rateList[1] 
    elif rv>0.50 and rv <= 0.75: # A
        myRating = rateList[2] 
    else:  # BBB
        myRating = rateList[3] 
    return myRating 

def assignDebtToAssetRatio(rating):
    if rating=='AAA': 
        debtToAssetRatio = np.random.uniform(0.40,0.45)         
    elif rating=='AA': 
        debtToAssetRatio = np.random.uniform(0.45,0.50)         
    elif rating=='A': 
        debtToAssetRatio = np.random.uniform(0.50,0.55)         
    else:  # BBB
        debtToAssetRatio = np.random.uniform(0.55,0.6)         
    return debtToAssetRatio 

def assignEquityVolatility(rating):
    if rating=='AAA': 
        equityVolatility = np.random.uniform(0.40,0.45)         
    elif rating=='AA': 
        equityVolatility = np.random.uniform(0.45,0.50)         
    elif rating=='A': 
        equityVolatility = np.random.uniform(0.50,0.55)         
    else:  # BBB
        equityVolatility = np.random.uniform(0.55,0.60)         
    return equityVolatility     

def getDelta(mu,sigma,dt,A,K):
    t1 = np.log(K/A)
    t2 = (mu-0.5*(sigma**2))*dt
    return np.divide(t1-t2,sigma*np.sqrt(dt))

def getD1(r,sigma,dt,A,K):
    t1 = math.log(A/K)
    t2 = (r+0.5*(sigma**2))*dt
    t3 = sigma*np.sqrt(dt)
    return np.divide(t1+t2,t3)   

def getE(r,sigma,dt,A,K):
    d1 = getD1(r,sigma,dt,A,K)
    d2 = d1 - sigma*math.sqrt(dt)
    return A*norm.cdf(d1)-np.exp(-r*dt)*K*norm.cdf(d2)

def getOptionDelta(r,sigma,dt,A,K):
    d1 = getD1(r,sigma,dt,A,K)
    optionDelta = norm.cdf(d1)
    return optionDelta 

def minimizeG(x,r,sigmaE,dt,E,K):
    A = x[0]
    if A<=0:
        return np.inf
    else:
        sigmaA = x[1]
        G1 = getE(r,sigmaA,dt,A,K)-E
        G2 = A*getOptionDelta(r,sigmaA,dt,A,K)*(sigmaA/sigmaE)-E
        return G1**2 + G2**2
    
def getVarA(mu,sigma,dt,A,K):
    t1 = np.exp((sigma**2)*dt)
    t2 = (A**2)*np.exp(2*mu*dt)
    return t2*(t1-1)

def getExpA(mu,dt,A):
    return np.multiply(A,np.exp(mu*dt))

def getCovAB(A,B,muA,muB,sigmaA,sigmaB,rhoAB,dt):
    t1 = (muA+muB)*dt
    t2 = sigmaA*sigmaB*rhoAB*dt    
    return A*B*np.exp(t1)*(np.exp(t2)-1)

def getCorAB(sigmaA,sigmaB,rhoAB,dt):
    num = np.exp(rhoAB*sigmaA*sigmaB*dt)-1
    tA = np.sqrt(np.exp((sigmaA**2)*dt)-1)
    tB = np.sqrt(np.exp((sigmaB**2)*dt)-1)
    return np.divide(num,tA*tB)   

def getDefaultCorAB(A,B,muA,muB,sigmaA,sigmaB,rhoAB,dt,KA,KB):
    dA = getDelta(muA,sigmaA,dt,A,KA)
    dB = getDelta(muB,sigmaB,dt,B,KB)     
    pA = norm.cdf(dA)
    pB = norm.cdf(dB)   
    pAB,err = mvn.mvnun(np.array([-100, -100]),np.array([dA, dB]),
                      np.array([0, 0]),np.array([[1,rhoAB],[rhoAB,1]]))   
    return np.divide(pAB-pA*pB,np.sqrt(pA*pB*(1-pA)*(1-pB)))


def getSigmaA(mu,sigma,dt,A,K):
    t1 = math.exp((sigma**2)*dt)
    t2 = A*math.exp(mu*dt)
    return t2*np.sqrt(t1-1)

def computeAssetValueMoments(N,A,mu,sigma,C,dt):
    hatA = np.zeros(N)
    Omega = np.zeros([N,N])
    for n in range(0,N):
        hatA[n] = getExpA(mu[n],dt,A[n])
        for m in range(0,N):
            Omega[n,m] = getCovAB(A[n],A[m],mu[n],mu[m],
                                  sigma[n],sigma[m],C[n,m],dt)
    return hatA,Omega

        

In [ ]:
import numpy as np
import math
import cmUtilities as util
import numpy.linalg as anp
import importlib 
from scipy.stats import norm
import scipy.integrate as nInt
import scipy

importlib.reload(util)

# -----------------------
# Beta-binomial model functions
# -----------------------

def betaMoment(a,b,momentNumber):
    num = math.gamma(a+momentNumber)*math.gamma(a+b)
    den = math.gamma(a+b+momentNumber)*math.gamma(a)
    f = np.divide(num,den)
    return f

def betaCalibrate(pTarget,rhoTarget):
    A = np.array([[rhoTarget,rhoTarget],[pTarget-1,pTarget]])
    a,b = np.dot(anp.inv(A),np.array([1-rhoTarget,0]))
    return a,b

def betaBinomialAnalytic(N,c,a,b,alpha):
    pmfBeta = np.zeros(N+1)
    den = util.computeBeta(a,b)
    for k in range(0,N+1):
        pmfBeta[k] =  util.getBC(N,k)*util.computeBeta(a+k,b+N-k)/den  
    cdfBeta = np.cumsum(pmfBeta)
    varAnalytic = c*np.interp(alpha,cdfBeta,np.linspace(0,N,N+1))
    esAnalytic = util.analyticExpectedShortfall(N,alpha,pmfBeta,c)
    return pmfBeta,cdfBeta,varAnalytic,esAnalytic
    
def betaBinomialSimulation(N,M,c,a,b,alpha):
    Z = util.generateGamma(a,b,M)
    pZ = np.transpose(np.tile(Z,(N,1)))
    U = np.random.uniform(0,1,[M,N])
    lossIndicator = 1*np.less(U,pZ)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es  

def betaLossDistribution(N,M,c,a,b,alpha):
    Z = util.generateGamma(a,b,M)
    pZ = np.transpose(np.tile(Z,(N,1)))
    U = np.random.uniform(0,1,[M,N])
    lossIndicator = 1*np.less(U,pZ)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    return lossDistribution  

# -----------------------
# Logit-Probit model functions
# -----------------------

def logitProbitMoment(z,mu,sigma,momentNumber,isLogit):
    if isLogit==1:
        v0 = (np.reciprocal(1+np.exp(-(mu+sigma*z))))**momentNumber
    else:
        v0 = (norm.cdf(mu+sigma*z))**momentNumber    
    density = util.gaussianDensity(z,0,1)
    f = v0*density
    return f  

def logitProbitCalibrate(x,pTarget,rhoTarget,isLogit):
    if x[1]<=0:
        return [100, 100]
    M1,err = nInt.quad(logitProbitMoment,-8,8,args=(x[0],x[1],1,isLogit)) 
    M2,err = nInt.quad(logitProbitMoment,-8,8,args=(x[0],x[1],2,isLogit)) 
    f1 = pTarget - M1    
    f2 = rhoTarget*(M1 - (M1**2)) - (M2 - (M1**2))    
    return [f1, f2]

def logitProbitBinomialAnalytic(N,c,mu,sigma,alpha,isLogit):
    pmf = np.zeros(N+1)
    for k in range(0,N+1):
        pmf[k],err=nInt.quad(logitProbitMixtureFunction,0,1,
                                          args=(N,k,mu,sigma,isLogit)) 
    cdf = np.cumsum(pmf)
    varAnalytic = c*np.interp(alpha,cdf,np.linspace(0,N,N+1))
    esAnalytic = util.analyticExpectedShortfall(N,alpha,pmf,c)
    return pmf,cdf,varAnalytic,esAnalytic

def logitProbitMixtureFunction(z,N,k,mu,sigma,isLogit):
    if isLogit==1:
        density = util.logitDensity(z,mu,sigma)
    else:
        density = util.probitDensity(z,mu,sigma)
    probTerm = util.getBC(N,k)*(z**k)*((1-z)**(N-k))
    f = probTerm*density
    return f    

def logitProbitBinomialSimulation(N,M,c,mu,sigma,alpha,isLogit):
    Z = np.random.normal(0,1,M)
    if isLogit==1:
        p = np.reciprocal(1+np.exp(-(mu+sigma*Z)))
    else:
        p = norm.cdf(mu+sigma*Z)
    pZ = np.transpose(np.tile(p,(N,1)))
    U = np.random.uniform(0,1,[M,N])
    lossIndicator = 1*np.less(U,pZ)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es    

# -----------------------
# Poisson-Gamma model functions
# -----------------------

def poissonGammaMoment(a,b,momentNumber):
    if momentNumber==1:
        myMoment = 1-np.power(np.divide(b,b+1),a)
    if momentNumber==2:
       v1 = np.power(np.divide(b,b+1),a)
       v2 = np.power(np.divide(b,b+2),a)
       myMoment = 1 - 2*v1 + v2
    return myMoment 

def poissonGammaCalibrate(x,pTarget,rhoTarget):
    if x[1]<=0:
        return [100, 100]
    M1 = poissonGammaMoment(x[0],x[1],1)
    M2 = poissonGammaMoment(x[0],x[1],2) 
    f1 = pTarget - M1    
    f2 = rhoTarget*(M1 - (M1**2)) - (M2 - (M1**2))    
    return [f1, f2]

def poissonGammaAnalytic(N,c,a,b,alpha):
    pmfPoisson = np.zeros(N+1)
    q = np.divide(b,b+1)
    den = math.gamma(a)
    for k in range(0,N+1):
        num = np.divide(math.gamma(a+k),scipy.misc.factorial(k))
        pmfPoisson[k] = np.divide(num,den)*np.power(q,a)*np.power(1-q,k)  
    cdfPoisson = np.cumsum(pmfPoisson)
    varAnalytic = c*np.interp(alpha,cdfPoisson,np.linspace(0,N,N+1))
    esAnalytic = util.analyticExpectedShortfall(N,alpha,pmfPoisson,c)
    return pmfPoisson,cdfPoisson,varAnalytic,esAnalytic

def poissonGammaSimulation(N,M,c,a,b,alpha):
    lam = np.random.gamma(a,1/b,M)
    H = np.zeros([M,N])
    for m in range(0,M):
        H[m,:] = np.random.poisson(lam[m],[N])
    lossIndicator = 1*np.greater_equal(H,1)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es  
 
# -----------------------
# Poisson-Mixture model functions
# -----------------------

def poissonMixtureMoment(s,a,b,momentNumber,whichModel):
    v0 = 1-np.exp(-s)
    if whichModel==0:
        myDensity = util.logNormalDensity(s,a,b)
    else:
        myDensity = util.weibullDensity(s,a,b)
    f = np.power(v0,momentNumber)*myDensity
    return f  

def poissonTransformedMixtureMoment(s,a,b,momentNumber,whichModel):
    vy = -np.log(1-s)
    jacobian = np.divide(1,1-s)
    if whichModel==0:
        psDensity = util.logNormalDensity(vy,a,b)*jacobian
    elif whichModel==1:
        psDensity = util.weibullDensity(vy,a,b)*jacobian
    myMoment = (s**momentNumber)*psDensity
    return myMoment  
      
def poissonMixtureAnalytic(N,myC,a,b,alpha,whichModel):
    pmfMixture = np.zeros(N+1)
    for k in range(0,N+1):
        pmfMixture[k],err = nInt.quad(poissonMixtureIntegral,0,k+1,
                                      args=(k,a,b,N,whichModel)) 
    cdfMixture = np.cumsum(pmfMixture)
    varAnalytic = myC*np.interp(alpha,cdfMixture,np.linspace(0,N,N+1))
    esAnalytic = util.analyticExpectedShortfall(N,alpha,pmfMixture,myC)
    return pmfMixture,cdfMixture,varAnalytic,esAnalytic

def poissonMixtureIntegral(s,k,a,b,N,whichModel):
    pDensity = util.poissonDensity(N*s,k)
    if whichModel==0:
        mixDensity = util.logNormalDensity(s,a,b)
    elif whichModel==1:
        mixDensity = util.weibullDensity(s,a,b)        
    f = pDensity*mixDensity
    return f

def poissonMixtureCalibrate(x,pTarget,rhoTarget,whichModel):
    if x[1]<=0:
        return [100, 100]
    M1,err = nInt.quad(poissonMixtureMoment,0.0001,0.9999,args=(x[0],x[1],1,whichModel)) 
    M2,err = nInt.quad(poissonMixtureMoment,0.0001,0.9999,args=(x[0],x[1],2,whichModel)) 
    f1 = pTarget - M1    
    f2 = rhoTarget*(M1 - (M1**2)) - (M2 - (M1**2))    
    return [f1, f2]

def poissonMixtureCalibrate1(x,pTarget,rhoTarget,whichModel):
    if x[1]<=0:
        return [100, 100]
    M1,err = nInt.quad(poissonTransformedMixtureMoment,0.0001,0.9999,args=(x[0],x[1],1,whichModel)) 
    M2,err = nInt.quad(poissonTransformedMixtureMoment,0.0001,0.9999,args=(x[0],x[1],2,whichModel)) 
    f1 = pTarget - M1    
    f2 = rhoTarget*(M1 - (M1**2)) - (M2 - (M1**2))    
    return [f1, f2]

def poissonMixtureSimulation(N,M,c,a,b,alpha,whichModel):
    if whichModel==0:
        lam = np.random.lognormal(a,b,M)
    elif whichModel==1:
         lam = b*np.random.weibull(a,M)
    H = np.zeros([M,N])
    for m in range(0,M):
        H[m,:] = np.random.poisson(lam[m],[N])
    lossIndicator = 1*np.greater_equal(H,1)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es  
      
# -----------------------
# CreditRisk+ model functions
# -----------------------

def crPlusOneFactor(N,M,w,p,c,v,alpha):
    S = np.random.gamma(v, 1/v, [M]) 
    wS =  np.transpose(np.tile(1-w + w*S,[N,1]))
    pS = np.tile(p,[M,1])*wS
    H = np.random.poisson(pS,[M,N])
    lossIndicator = 1*np.greater_equal(H,1)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es          

def calibrateBeta(myG,myXi):
    a1 = np.divide(myG*(1-myXi),myXi)
    b1 = np.divide((1-myG)*(1-myXi),myXi)
    return a1, b1

def crPlusOneFactorLGD(N,M,w,p,c,v,gBar,xi,alpha):
    a1,b1 = calibrateBeta(gBar,xi)
    LGD = np.random.beta(a1,b1,[M,N])
    S = np.random.gamma(v, 1/v, [M]) 
    wS =  np.transpose(np.tile(1-w + w*S,[N,1]))
    pS = np.tile(p,[M,1])*wS
    H = np.random.poisson(pS,[M,N])
    lossIndicator = 1*np.greater_equal(H,1)
    lossDistribution = np.sort(np.dot(LGD*lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es   

def crPlusMultifactor(N,M,wMat,p,c,aVec,alpha,rId):
    K = len(aVec)
    S = np.zeros([M,K])
    for k in range(0,K):        
        S[:,k] = np.random.gamma(aVec[k], 1/aVec[k], [M]) 
    W = wMat[rId,:]
    # Could replace tile with np.kron(W[:,0],np.ones([1,M])), but it's slow
    wS =  np.tile(W[:,0],[M,1]) + np.dot(S,np.transpose(W[:,1:]))
    pS = np.tile(p,[M,1])*wS
    H = np.random.poisson(pS,[M,N])
    lossIndicator = 1*np.greater_equal(H,1)
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es          
      
# -----------------------
# Miscellaneous functions
# -----------------------

def calibrateVerify(a,b,targetP,targetRho,model):
    if (model==0) | (model==4): # Binomial and Poisson
        M1 = targetP
        M2 = targetP**2
    if model==1: # Beta-binomial
        M1 = betaMoment(a,b,1)
        M2 = betaMoment(a,b,2)
    elif model==2: # Logit
        M1,err = nInt.quad(logitProbitMoment,-8,8,args=(a,b,1,1)) 
        M2,err = nInt.quad(logitProbitMoment,-8,8,args=(a,b,2,1)) 
    elif model==3: # Probit
        M1,err = nInt.quad(logitProbitMoment,-8,8,args=(a,b,1,0)) 
        M2,err = nInt.quad(logitProbitMoment,-8,8,args=(a,b,2,0)) 
    elif model==5: # Basic Poisson-gamma
        M1 = poissonGammaMoment(a,b,1)
        M2 = poissonGammaMoment(a,b,2)
    elif model==6: # Poisson log-normal
        M1,err = nInt.quad(poissonMixtureMoment,0,1,args=(a,b,1,0)) 
        M2,err = nInt.quad(poissonMixtureMoment,0,1,args=(a,b,2,0)) 
    elif model==7: # Poisson Weibull
        M1,err = nInt.quad(poissonMixtureMoment,0,1,args=(a,b,1,1)) 
        M2,err = nInt.quad(poissonMixtureMoment,0,1,args=(a,b,2,1))         
    print("Target default probability is %0.3f and target default correlation is %0.3f" % (targetP,targetRho))
    print("Calibrated default probability is %0.3f and calibrated default correlation is %0.3f" % (M1,defCorr(M1,M2)))
    return M1,M2

def defCorr(M1,M2):
    return np.divide(M2-M1**2,M1-M1**2)

In [ ]:
import numpy as np
import math
import cmUtilities as util
import assetCorrelation as ac
import importlib 
from scipy.stats import norm
import scipy.integrate as nInt
from scipy.stats import t as myT
import numpy.linalg as anp
import scipy

from rpy2.robjects.packages import importr
gig = importr('GIGrvg')

importlib.reload(util)

def getY(N,M,p,rho,nu,isT):
    G = np.transpose(np.tile(np.random.normal(0,1,M),(N,1)))
    e = np.random.normal(0,1,[M,N])
    if isT==1:
        W = np.transpose(np.sqrt(nu/np.tile(np.random.chisquare(nu,M),(N,1))))
        Y = np.multiply(W,math.sqrt(rho)*G + math.sqrt(1-rho)*e)
    else:
        Y = math.sqrt(rho)*G + math.sqrt(1-rho)*e
    return Y   

def getGaussianY(N,M,p,rho):
    G = np.transpose(np.tile(np.random.normal(0,1,M),(N,1)))
    e = np.random.normal(0,1,[M,N])
    Y = math.sqrt(rho)*G + math.sqrt(1-rho)*e
    return Y   

def getTY(N,M,p,rho,nu):
    G = np.transpose(np.tile(np.random.normal(0,1,M),(N,1)))
    e = np.random.normal(0,1,[M,N])
    W = np.transpose(np.sqrt(nu/np.tile(np.random.chisquare(nu,M),(N,1))))
    Y = np.multiply(W,math.sqrt(rho)*G + math.sqrt(1-rho)*e)
    return Y   

def calibrateGaussian(x,myP,targetRho):
    jointDefaultProb = ac.jointDefaultProbability(myP,myP,x)
    defaultCorrelation = np.divide(jointDefaultProb-myP**2,myP*(1-myP))
    return np.abs(defaultCorrelation-targetRho)

def getY2r(N,M,p,myRho,rId,nu,P,isT):
    rhoVector = myRho[rId]
    rhoMatrix = np.tile(rhoVector,(M,1))
    G = np.transpose(np.tile(np.random.normal(0,1,M),(N,1)))
    e = np.random.normal(0,1,[M,N])
    systematic = np.multiply(np.sqrt(rhoMatrix),G)
    idiosyncratic = np.multiply(np.sqrt(1-rhoMatrix),e)
    if isT==1:
        W = np.transpose(np.sqrt(nu/np.tile(np.random.chisquare(nu,M),(N,1))))
        Y = np.multiply(W,systematic + idiosyncratic)
    else:
        Y = systematic + idiosyncratic
    return Y    

def oneFactorGaussianModel(N,M,p,c,rho,alpha):
    Y = getGaussianY(N,M,p,rho)
    K = norm.ppf(p)*np.ones((M,1))        
    lossIndicator = 1*np.less(Y,K)     
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es      

def oneFactorTModel(N,M,p,c,rho,nu,alpha):
    Y = getTY(N,M,p,rho,nu)
    K = myT.ppf(p,nu)*np.ones((M,1))        
    lossIndicator = 1*np.less(Y,K)     
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es      

def oneFactorThresholdModel(N,M,p,c,rho,nu,alpha,isT):
    Y = getY(N,M,p,rho,nu,isT)
    if isT==1:
        K = myT.ppf(p,nu)*np.ones((M,1))        
    else:
        K = norm.ppf(p)*np.ones((M,1))        
    lossIndicator = 1*np.less(Y,K)     
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es      

def oneFactorThresholdLossDistribution(N,M,p,c,rho,nu,alpha,isT):
    Y = getY(N,M,p,rho,nu,isT)
    if isT==1:
        K = myT.ppf(p,nu)*np.ones((M,1))        
    else:
        K = norm.ppf(p)*np.ones((M,1))        
    lossIndicator = 1*np.less(Y,K)     
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    return lossDistribution      
    
def asrfModel(myP,rho,c,alpha):
    myX = np.linspace(0.0001,0.9999,100)
    num = np.sqrt(1-rho)*norm.ppf(myX)-norm.ppf(myP)
    cdf = norm.cdf(num/np.sqrt(rho))
    pdf = util.asrfDensity(myX,myP,rho)
    varAnalytic = np.sum(c)*np.interp(alpha,cdf,myX)
    esAnalytic = asrfExpectedShortfall(alpha,myX,cdf,pdf,c,rho,myP)
    return pdf,cdf,varAnalytic,esAnalytic
    
def asrfExpectedShortfall(alpha,myX,cdf,pdf,c,rho,myP):
    expectedShortfall = np.zeros(len(alpha))
    for n in range(0,len(alpha)):   
        myAlpha = np.linspace(alpha[n],1,1000)
        loss = np.sum(c)*np.interp(myAlpha,cdf,myX)
        prob = np.interp(loss,myX,pdf)
        expectedShortfall[n] = np.dot(loss,prob)/np.sum(prob)
    return expectedShortfall    
        
def asrfMoment(x,p,rho,whichMoment):
    if whichMoment==1:    
        f = x*util.asrfDensity(x,p,rho)
    elif whichMoment==2:
        f = np.power(x,2)*util.asrfDensity(x,p,rho)
    return f
    
def getAsrfMoments(p,rho):
    el,err = nInt.quad(asrfMoment,0,1,args=(p,rho,1)) 
    M2,err = nInt.quad(asrfMoment,0,1,args=(p,rho,2)) 
    ul = np.sqrt(M2 - np.power(el,2))
    return el, ul    

def computeP_t(p,rho,y,w,nu):
    num = np.sqrt(w/nu)*myT.ppf(p,nu)-np.multiply(np.sqrt(rho),y)
    pZ = norm.cdf(np.divide(num,np.sqrt(1-rho)))
    return pZ

def computeP(p,rho,g):
    num = norm.ppf(p)-np.multiply(np.sqrt(rho),g)
    pG = norm.cdf(np.divide(num,np.sqrt(1-rho)))
    return pG

def jointDefaultProbabilityT(p,q,myRho,nu):
    lowerBound = np.maximum(nu-40,2)
    support = [[-10,10],[lowerBound,nu+40]]
    pr,err=nInt.nquad(jointIntegrandT,support,args=(p,q,myRho,nu))
    return pr

def jointDefaultProbabilityG(p,q,myRho):
    pr,err=nInt.quad(jointIntegrandG,-10,10,args=(p,q,myRho))
    return pr

def jointIntegrandT(g,w,p,q,myRho,nu):
    p1 = computeP_t(p,myRho,g,w,nu)
    p2 = computeP_t(q,myRho,g,w,nu)
    density1 = util.gaussianDensity(g,0,1)
    density2 = util.chi2Density(w,nu)    
    f = p1*p2*density1*density2
    return f

def jointIntegrandG(g,p,q,myRho):
    p1 = computeP(p,myRho,g)
    p2 = computeP(q,myRho,g)
    density = util.gaussianDensity(g,0,1)
    f = p1*p2*density
    return f

def bivariateTDensity(x1,x2,rho,nu,d=2):
    Sigma = np.array([[1,rho],[rho,1]])
    myX = np.array([x1,x2])
    t1 = math.gamma((nu+d)/2)
    t2 = math.gamma(nu/2) 
    t3 = np.power(nu*math.pi,d/2)
    t4 = np.sqrt(anp.det(Sigma))
    constant = np.divide(t1,t2*t3*t4)
    t5 = np.dot(np.dot(myX,anp.inv(Sigma)),myX)
    integrand = constant*np.power(1+t5/nu,-(nu+d)/2)
    return integrand

def bivariateTCdf(yy,xx,rho,nu):    
    t_ans, err = nInt.dblquad(bivariateTDensity, -10, xx,
                   lambda x: -10,
                   lambda x: yy,args=(rho,nu))
    return t_ans

def bivariateGDensity(x1,x2,rho):
    S = np.array([[1,rho],[rho,1]])
    t1 = 2*math.pi*np.sqrt(anp.det(S))
    t2 = np.dot(np.dot(np.array([x1,x2]),anp.inv(S)),np.array([x1,x2]))
    return np.divide(1,t1)*np.exp(-t2/2)


def buildAssetCorrelationMatrix(a,b,regionId):
    J = len(b)
    R = np.zeros([J,J])
    for n in range(0,J):
        for m in range(0,J):
            if regionId[n]==regionId[m]:
                R[n,m] = a + (1-a)*np.sqrt(b[n]*b[m])
            else:
                R[n,m] = a
    return R

def buildDefaultCorrelationMatrix(a,b,pMean,regionId,nu):
    J = len(regionId)
    R = buildAssetCorrelationMatrix(a,b,regionId)    
    D = np.zeros([J,J])
    for n in range(0,J):
        p_n = pMean[n]
        for m in range(0,J):
            p_m = pMean[m]
            p_nm = bivariateTCdf(norm.ppf(p_n),norm.ppf(p_m),R[n,m],nu)
            D[n,m] = (p_nm - p_n*p_m)/math.sqrt(p_n*(1-p_n)*p_m*(1-p_m))
    return D

def calibrateOF(x,B,pMean,regionId,nu):
    a = x[0]
    b = np.array([x[1],x[2],x[3]])
    D = buildDefaultCorrelationMatrix(a,b,pMean,regionId,nu)     
    f = anp.norm(D-B,ord='fro')
    return f

def calibrateMFT(B,pMean,regionId,nu):
    myBounds = ((0.001,0.30),(0.001,0.30),(0.001,0.30),
                (0.001,0.30))                            
    M = 100
    xRandom = np.random.uniform(0,0.30,[M,4])
    functionValues = np.zeros(M)
    for m in range(0,M):
        functionValues[m] = calibrateOF(xRandom[m,:],B,pMean,regionId,nu)
    newOF=np.min(functionValues)
    xStart = xRandom[functionValues==newOF]
    xhat = scipy.optimize.minimize(calibrateOF, 
                    xStart, args=(B,pMean,regionId,nu), 
                    method='SLSQP', jac=None, bounds=myBounds)   
    return xhat    

def calibrateT(x,myP,targetRho,nu):
    jointDefaultProb = jointDefaultProbabilityT(myP,myP,x,nu)
    defaultCorrelation = np.divide(jointDefaultProb-myP**2,myP*(1-myP))
    return np.abs(defaultCorrelation-targetRho)

def thresholdCalibrationGridSearch(dGrid,myP,rhoTarget,whichModel,nu=30):
    jointDefaultProb = np.zeros([2,25])
    dEstimate = np.zeros([2,25])
    for n in range(0,len(dGrid)):
        print("Iteration %d" % (n+1))
        if whichModel==1:
            support = [[-8,norm.ppf(myP)],[-8,norm.ppf(myP)]]
            jointDefaultProb[0,n] = jointDefaultProbabilityG(myP,myP,dGrid[n])
            jointDefaultProb[1,n],err = nInt.nquad(bivariateGDensity,support,args=(dGrid[n],2))
        elif whichModel==2:   
            support = [[-8,myT.ppf(myP,nu)],[-8,myT.ppf(myP,nu)]]
            jointDefaultProb[0,n] = jointDefaultProbabilityT(myP,myP,dGrid[n],nu)
            jointDefaultProb[1,n],err = nInt.nquad(bivariateTDensity,support,args=(dGrid[n],nu))
        dEstimate[0,n] = np.divide(jointDefaultProb[0,n]-myP**2,myP*(1-myP))
        dEstimate[1,n] = np.divide(jointDefaultProb[1,n]-myP**2,myP*(1-myP))
    print("The conditonal approach gives %0.2f" % (np.interp(rhoTarget,dEstimate[0,:],dGrid)))
    print("The classic approach gives %0.2f" % (np.interp(rhoTarget,dEstimate[1,:],dGrid)))
    return dEstimate

def tTailDependenceCoefficient(rho,nu):
    a = -np.sqrt(np.divide((nu+1)*(1-rho),1+rho)) 
    tCoefficient = 2*myT.cdf(a,nu+1)    
    return tCoefficient

def tCalibrate(x,myP,rhoTarget,tdTarget):
    if (x[0]<=0) | (x[1]<=0):
        return [100, 100]
    jointDefaultProb = jointDefaultProbabilityT(myP,myP,x[0],x[1])
    rhoValue = np.divide(jointDefaultProb-myP**2,myP*(1-myP))
    tdValue = tTailDependenceCoefficient(x[0],x[1])
    f1 = rhoValue - rhoTarget    
    f2 = tdValue - tdTarget    
    return [f1, f2]

def getMultiFactorY(N,M,p,a,b,rId,nu,isT):
    G = np.transpose(np.tile(np.random.normal(0,1,M),(N,1)))
    regions = np.random.normal(0,1,[M,len(np.unique(rId))]) 
    e = np.random.normal(0,1,[M,N])
    R = regions[:,rId]
    A = np.tile(a*np.ones(N),(M,1))
    B = np.tile(b[rId],(M,1))
    T0 = np.multiply(np.sqrt(A),G)
    T1 = np.sqrt(1-A)
    T2 = np.multiply(np.sqrt(B),R) + np.multiply(np.sqrt(1-B),e)
    if isT==1: 
        W = np.transpose(np.sqrt(nu/np.tile(np.random.chisquare(nu,M),(N,1))))
        return np.multiply(W,T0+np.multiply(T1,T2))
    else: 
        return T0+np.multiply(T1,T2)

def multiFactorThresholdModel(N,M,a,b,rId,p,c,nu,alpha,isT):
    Y = getMultiFactorY(N,M,p,a,b,rId,nu,isT)
    if isT==1:
        K = myT.ppf(p,nu)*np.ones((M,1)) 
    else:
        K = norm.ppf(p)*np.ones((M,1))        
    lossIndicator = 1*np.less(Y,K)     
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es      

def nvmDensity(v,x,myA,whichModel):
    t1 = np.divide(1,np.sqrt(2*math.pi*v))
    t2 = np.exp(-np.divide(x**2,2*v))
    if whichModel==0:
        return t1*t2*util.gammaDensity(v,myA,myA)
    elif whichModel==1:
        return t1*t2*util.gigDensity(v,myA)
    
def nvmPdf(x,myA,whichModel):
    f,err = nInt.quad(nvmDensity,0,50,args=(x,myA,whichModel)) 
    return f

def nvmCdf(x,myA,whichModel):
    F,err = nInt.quad(nvmPdf,-8,x,args=(myA,whichModel)) 
    return F    

def nvmTarget(x,myVal,myA,whichModel):
    F,err = nInt.quad(nvmPdf,-8,x,args=(myA,whichModel)) 
    return F-myVal

def nvmPpf(myVal,myA,whichModel):
    r = scipy.optimize.fsolve(nvmTarget,0,args=(myVal,myA,whichModel))
    return r[0]        

def getNVMY(N,M,rho,myA,whichModel):
    G = np.transpose(np.tile(np.random.normal(0,1,M),(N,1)))
    e = np.random.normal(0,1,[M,N])
    if whichModel==0:
        V = np.transpose(np.sqrt(np.tile(np.random.gamma(myA,1/myA,M),(N,1))))
    elif whichModel==1:
        V = np.transpose(np.sqrt(np.tile(gig.rgig(M,1,myA,myA),(N,1))))
    Y = np.multiply(V,math.sqrt(rho)*G + math.sqrt(1-rho)*e)
    return Y   

def computeP_NVM(p,rho,y,v,myA,invCdf):
    num = np.sqrt(1/v)*invCdf-np.multiply(np.sqrt(rho),y)
    pZ = norm.cdf(np.divide(num,np.sqrt(1-rho)))
    return pZ
  
def nvmKurtosis(rho,myA,whichModel):
    if whichModel==0:
        return 3*(1+myA)/myA
    elif whichModel==1:
        num = scipy.special.kn(3, myA)*scipy.special.kn(1, myA)
        den = scipy.special.kn(2, myA)**2
        return 3*np.divide(num,den)  
       
def ghVariance(myA):
    return  scipy.special.kn(2, myA)/scipy.special.kn(1, myA)   
        
def jointDefaultProbabilityNVM(p,q,myRho,myA,whichModel):    
    invCdf = nvmPpf(p,myA,whichModel)
    support = [[-8,8],[0,100]]
    pr,err=nInt.nquad(jointIntegrandNVM,support,args=(p,q,myRho,myA,invCdf,whichModel))
    return pr

def jointIntegrandNVM(g,v,p,q,myRho,myA,invCdf,whichModel):
    p1 = computeP_NVM(p,myRho,g,v,myA,invCdf)
    p2 = computeP_NVM(q,myRho,g,v,myA,invCdf)
    density1 = util.gaussianDensity(g,0,1)
    if whichModel==0:
        density2 = util.gammaDensity(v,myA,myA)            
    elif whichModel==1:
        density2 = util.gigDensity(v,myA)    
    return p1*p2*density1*density2         
        
def nvmCalibrate(x,myP,rhoTarget,kTarget,whichModel):
    if (x[0]<=0) | (x[1]<=0):
        return [100, 100]
    jointDefaultProb = jointDefaultProbabilityNVM(myP,myP,x[0],x[1],whichModel)
    rhoValue = np.divide(jointDefaultProb-myP**2,myP*(1-myP))
    kValue = nvmKurtosis(x[0],x[1],whichModel)
    f1 = rhoValue - rhoTarget    
    f2 = kValue - kTarget    
    return [f1, f2]

def oneFactorNVMModel(N,M,p,c,rho,myA,alpha,whichModel):
    Y = getNVMY(N,M,rho,myA,whichModel)
    invVector = np.zeros(N)
    for n in range(0,N): 
        invVector[n] = nvmPpf(p[n],myA,whichModel)    
    K = invVector*np.ones((M,1))        
    lossIndicator = 1*np.less(Y,K)     
    lossDistribution = np.sort(np.dot(lossIndicator,c),axis=None)
    el,ul,var,es=util.computeRiskMeasures(M,lossDistribution,alpha)
    return el,ul,var,es      



In [ ]:
import numpy as np
import math
import scipy
import cmUtilities as util
import binomialPoissonModels as bp
import importlib 
from scipy.stats import norm
import scipy.integrate as nInt
import thresholdModels as th
from scipy.stats import t as myT

importlib.reload(util)
importlib.reload(th)
importlib.reload(bp)

def mcThresholdTDecomposition(N,M,S,p,c,rho,nu,isT,myAlpha):
    contributions = np.zeros([N,S,2])
    var = np.zeros(S)
    es = np.zeros(S)
    K = myT.ppf(p,nu)*np.ones((M,1))        
    for s in range(0,S):
        print("Iteration: %d" % (s+1))
        Y = th.getY(N,M,p,rho,nu,isT)
        myD = 1*np.less(Y,K)     
        myLoss = np.sort(np.dot(myD,c),axis=None)
        el,ul,var[s],es[s]=util.computeRiskMeasures(M,myLoss,np.array([myAlpha]))
        varVector = c*myD[np.dot(myD,c)==var[s],:]
        esVector = c*myD[np.dot(myD,c)>=var[s],:]
        contributions[:,s,0] = np.sum(varVector,0)/varVector.shape[0]
        contributions[:,s,1] = np.sum(esVector,0)/esVector.shape[0]
    return contributions,var,es

def mcThresholdGDecomposition(N,M,S,p,c,rho,nu,isT,myAlpha):
    contributions = np.zeros([N,S,2])
    var = np.zeros(S)
    es = np.zeros(S)
    K = norm.ppf(p)*np.ones((M,1))        
    for s in range(0,S):
        print("Iteration: %d" % (s+1))
        Y = th.getY(N,M,p,rho,nu,isT)
        myD = 1*np.less(Y,K)     
        myLoss = np.sort(np.dot(myD,c),axis=None)
        el,ul,var[s],es[s]=util.computeRiskMeasures(M,myLoss,np.array([myAlpha]))
        varVector = c*myD[np.dot(myD,c)==var[s],:]
        esVector = c*myD[np.dot(myD,c)>=var[s],:]
        contributions[:,s,0] = np.sum(varVector,0)/varVector.shape[0]
        contributions[:,s,1] = np.sum(esVector,0)/esVector.shape[0]
    return contributions,var,es


def mcThresholdIndDecomposition(N,M,S,p,c,myAlpha):
    contributions = np.zeros([N,S,2])
    var = np.zeros(S)
    es = np.zeros(S)
    for s in range(0,S):
        print("Iteration: %d" % (s+1))
        myLoss,myD = bp.independentBinomialLossDistribution(N,M,p,c,myAlpha,1)
        el,ul,var[s],es[s]=util.computeRiskMeasures(M,myLoss,np.array([myAlpha]))
        varVector = c*myD[np.dot(myD,c)==var[s],:]
        esVector = c*myD[np.dot(myD,c)>=var[s],:]
        contributions[:,s,0] = np.sum(varVector,0)/varVector.shape[0]
        contributions[:,s,1] = np.sum(esVector,0)/esVector.shape[0]
    return contributions,var,es

def computeMGF(t,p,c): 
    return 1-p+p*np.exp(c*t)
    
def computeCGF(t,p,c):
    return np.sum(np.log(computeMGF(t,p,c)))
 
def computeCGF_1(t,p,c):
    num = c*p*np.exp(c*t)
    den = computeMGF(t,p,c)
    return np.sum(np.divide(num,den))

def computeCGF_2(t,p,c,asVector=0):
    num = (1-p)*(c**2)*p*np.exp(c*t)
    den = np.power(computeMGF(t,p,c),2)
    if asVector==1:
        return np.divide(num,den)
    else:
        return np.sum(np.divide(num,den))

def computeCGF_3(t,p,c):
    num1 = (1-p)*(c**3)*p*np.exp(c*t)
    num2 = 2*(1-p)*(c**3)*(p**2)*np.exp(2*c*t)
    den1 = np.power(computeMGF(t,p,c),2)
    den2 = np.power(computeMGF(t,p,c),3)
    return np.sum(np.divide(num1,den1)-np.divide(num2,den2))

def getSaddlePoint(p,c,l,startPoint=0.00025):
    r = scipy.optimize.root(computeCGFRoot,startPoint,args=(p,c,l),method='hybr')
    return r.x    

def step(x):
    return 1 * (x > 0)

def computeCGFRoot(t,p,c,l):
    return computeCGF_1(t,p,c)-l

def getJ(l,p,c,t_l,myOrder):
    K2 = computeCGF_2(t_l,p,c)
    if myOrder==0:
        return np.sqrt(np.divide(1,2*math.pi*K2))
    if myOrder==1:
        t0 = K2*(t_l**2)
        return np.sign(t_l)*np.exp(0.5*t0)*norm.cdf(-np.sqrt(t0))
    if myOrder==2:
        return K2*(getJ(l,p,c,t_l,0)-t_l*getJ(l,p,c,t_l,1))

def saddlePointDensity(l,p,c):
    t_l = getSaddlePoint(p,c,l)
    return np.exp(computeCGF(t_l,p,c)-t_l*l)*getJ(l,p,c,t_l,0)

def saddlePointTailProbability(l,p,c):
    t_l = getSaddlePoint(p,c,l)
    return np.exp(computeCGF(t_l,p,c)-t_l*l)*getJ(l,p,c,t_l,1)
    
def saddlePointShortfallIntegral(l,p,c):
    den = saddlePointTailProbability(l,p,c)
    t_l = getSaddlePoint(p,c,l)
    return l + np.exp(computeCGF(t_l,p,c)-t_l*l)*getJ(l,p,c,t_l,2)/den
    
def identifyVaRInd(x,p,c,myAlpha):
    tpY = saddlePointTailProbability(x,p,c)
    return 1e4*np.power((1-tpY)-myAlpha,2)

def getVaRC(l,p,c):
    t_l = getSaddlePoint(p,c,l)
    return np.divide(c*p*np.exp(c*t_l),computeMGF(t_l,p,c))
    
def getESC(l,p,c):
    varPart = getVaRC(l,p,c)
    myAlpha = saddlePointTailProbability(l,p,c)
    t_l = getSaddlePoint(p,c,l)
    K2 = computeCGF_2(t_l,p,c)
    myW = computeCGF_2(t_l,p,c,1)
    t0 = np.exp(computeCGF(t_l,p,c)-t_l*l)*getJ(l,p,c,t_l,2)
    return varPart + np.divide(t0*np.divide(myW,K2),myAlpha)    

def saddlePointApprox(l,p,c,t_l,myDegree,constant=0):
    if myDegree==1:
        constant = step(-t_l)
    elif myDegree==2:
        constant = step(-t_l)*(np.dot(p,c)-l)
    coefficient = np.exp(computeCGF(t_l,p,c)-t_l*l)
    return  constant + coefficient*getJ(l,p,c,t_l,myDegree)
    
def getPy(p,y,p1,p2,whichModel,v=0):
    if whichModel==0: # Gaussian threshold
        return th.computeP(p,p1,y)
    elif whichModel==1: # beta
        return y*np.ones(len(p))
    elif whichModel==2: # CreditRisk+
        v = p*(1-p1+p1*y)
        return np.maximum(np.minimum(1-np.exp(-v),0.999),0.0001)
    elif whichModel==3: # logit
        return np.reciprocal(1+np.exp(-(p1+p2*y)))
    elif whichModel==4: # probit
        return norm.ppf(p1+p2*y)    
    elif whichModel==5: # Weibull
        return np.maximum(np.minimum(1-np.exp(-y),0.999),0.0001)*np.ones(len(p))
    if whichModel==6: # t threshold
        return th.computeP_t(p,p1,y,v,p2)
    
def getYDensity(y,p1,p2,whichModel,v=0):
    if whichModel==0: 
        return util.gaussianDensity(y,0,1)
    elif whichModel==1:
        return util.betaDensity(y,p1,p2)
    elif whichModel==2:
        return util.gammaDensity(y,p2,p2)    
    elif whichModel==3:
        return util.gaussianDensity(y,0,1)
    elif whichModel==4:
        return util.gaussianDensity(y,0,1)
    elif whichModel==5:
        return util.weibullDensity(y,p1,p2)   
    elif whichModel==6:
        return util.gaussianDensity(y,0,1)*util.chi2Density(v,p2)   

def computeYIntegral(y,l,p,c,p1,p2,whichModel,myDegree):
    pY = getPy(p,y,p1,p2,whichModel)
    d = getYDensity(y,p1,p2,whichModel)
    t_l = getSaddlePoint(pY,c,l)
    return saddlePointApprox(l,pY,c,t_l,myDegree)*d  

def identifyVaR(x,p,c,p1,p2,whichModel,myAlpha):
    tpY = myApprox(x,p,c,p1,p2,whichModel,1)
    return 1e6*np.power((1-tpY)-myAlpha,2)

def getIntegrationBounds(whichModel):
    if whichModel==0:
        lB,uB = -8,8
    elif whichModel==1:
        lB,uB = 0.0001,0.9999
    elif whichModel==2:
        lB,uB = 0.0001,100
    if whichModel==3:
        lB,uB = -8,8
    if whichModel==4:
        lB,uB = -8,8
    elif whichModel==5:
        lB,uB = 0.0001,35
    return lB,uB

def myApprox(l,p,c,p1,p2,whichModel,myDegree,constant=0,den=1):
    lB,uB = getIntegrationBounds(whichModel)
    if myDegree==2:
        constant = l
        den,err = nInt.quad(computeYIntegral,lB,uB,args=(l,p,c,p1,p2,whichModel,1))        
    num,err = nInt.quad(computeYIntegral,lB,uB,args=(l,p,c,p1,p2,whichModel,myDegree))
    return constant + np.divide(num,den) 

def varCNumerator(y,l,myN,p,c,p1,p2,whichModel,v=0):
    pY = getPy(p,y,p1,p2,whichModel,v)
    d = getYDensity(y,p1,p2,whichModel,v)
    t_l = getSaddlePoint(pY,c,l)
    num = pY[myN]*np.exp(c[myN]*t_l)
    den = computeMGF(t_l,pY[myN],c[myN])
    return np.divide(num,den)*saddlePointApprox(l,pY,c,t_l,0)*d

def myVaRCY(l,p,c,p1,p2,whichModel):
    lB,uB = getIntegrationBounds(whichModel)
    den = myApprox(l,p,c,p1,p2,whichModel,0)
    num = np.zeros(len(p))
    for n in range(0,len(p)):
        num[n],err = nInt.quad(varCNumerator,lB,uB,args=(l,n,p,c,p1,p2,whichModel))
    return c*np.divide(num,den)

def esCVaR(y,l,myN,p,c,p1,p2,whichModel,myAlpha,extraTerm=0):
    pY = getPy(p,y,p1,p2,whichModel)
    d = getYDensity(y,p1,p2,whichModel)
    t_l = getSaddlePoint(pY,c,l)
    baseTerm = np.divide(pY[myN]*np.exp(c[myN]*t_l),computeMGF(t_l,pY[myN],c[myN]))
    if t_l<0:
        extraTerm = (pY[myN]-baseTerm)/myAlpha
    return (baseTerm+extraTerm)*d 

def myESCY(l,p,c,p1,p2,whichModel):
    lB,uB = getIntegrationBounds(whichModel)
    myAlpha = myApprox(l,p,c,p1,p2,whichModel,1)
    esC = np.zeros(len(p))
    for n in range(0,len(p)):
        esC[n],err = nInt.quad(integrateAll,lB,uB,args=(l,n,p,c,p1,p2,whichModel))
    return (1/myAlpha)*esC

def integrateAll(y,l,n,p,c,p1,p2,whichModel):
    pY = getPy(p,y,p1,p2,whichModel)
    d = getYDensity(y,p1,p2,whichModel)
    t_l = getSaddlePoint(pY,c,l)
    varPart = getVaRPart(l,n,pY,c,t_l)
    esPart = getESPart(l,n,pY,c,t_l)
    correctPart = getCorrectionPart(l,n,pY,c,t_l)
    return (varPart + esPart + correctPart)*d

def getVaRPart(l,myN,pY,c,t_l):
    baseTerm = np.divide(pY[myN]*np.exp(c[myN]*t_l),computeMGF(t_l,pY[myN],c[myN]))
    return c[myN]*baseTerm*saddlePointApprox(l,pY,c,t_l,1)

def getESPart(l,myN,pY,c,t_l):
    K2 = computeCGF_2(t_l,pY,c)
    myW = computeCGF_2(t_l,pY,c,1)
    t0 = np.exp(computeCGF(t_l,pY,c)-t_l*l)*getJ(l,pY,c,t_l,2)
    return np.divide(t0*np.divide(myW[myN],K2),1)    

def getCorrectionPart(l,myN,pY,c,t_l):
    t0 = c[myN]*pY[myN]
    t1 = computeCGF_1(t_l,pY[myN],c[myN])
    return step(-t_l)*(t0-t1)

def computeYIntegralT(y,v,l,p,c,p1,p2,whichModel,myDegree):
    pY = getPy(p,y,p1,p2,whichModel,v)
    d = getYDensity(y,p1,p2,whichModel,v)
    t_l = getSaddlePoint(pY,c,l)
    return saddlePointApprox(l,pY,c,t_l,myDegree)*d  

def myApproxT(l,p,c,p1,p2,whichModel,myDegree,constant=0,den=1):
    lowerBound = np.maximum(p2-20,0.0001)
    support = [[-8,8],[lowerBound,p2+8]]
    if myDegree==2:
        constant = l
        den,err = nInt.nquad(computeYIntegralT,support,args=(l,p,c,p1,p2,whichModel,1))        
    num,err = nInt.nquad(computeYIntegralT,support,args=(l,p,c,p1,p2,whichModel,myDegree))
    return constant + np.divide(num,den) 

def identifyVaRT(x,p,c,p1,p2,whichModel,myAlpha):
    tpY = myApproxT(x,p,c,p1,p2,whichModel,1)
    return 1e6*np.power((1-tpY)-myAlpha,2)

def varCNumeratorT(y,v,l,myN,p,c,p1,p2,whichModel):
    pY = getPy(p,y,p1,p2,whichModel,v)
    d = getYDensity(y,p1,p2,whichModel,v)
    t_l = getSaddlePoint(pY,c,l)
    num = pY[myN]*np.exp(c[myN]*t_l)
    den = computeMGF(t_l,pY[myN],c[myN])
    return np.divide(num,den)*saddlePointApprox(l,pY,c,t_l,0)*d

def myVaRCYT(l,p,c,p1,p2,whichModel):
    lowerBound = np.maximum(p2-20,0.0001)
    support = [[-8,8],[lowerBound,p2+8]]
    den = myApproxT(l,p,c,p1,p2,whichModel,0)
    num = np.zeros(len(p))
    for n in range(0,len(p)):
        num[n],err = nInt.nquad(varCNumeratorT,support,args=(l,n,p,c,p1,p2,whichModel))
    return c*np.divide(num,den)

def myESCYT(l,p,c,p1,p2,whichModel):
    lowerBound = np.maximum(p2-20,0.0001)
    support = [[-8,8],[lowerBound,p2+8]]
    myAlpha = myApproxT(l,p,c,p1,p2,whichModel,1)
    esC = np.zeros(len(p))
    for n in range(0,len(p)):
        esC[n],err = nInt.nquad(integrateAllT,support,args=(l,n,p,c,p1,p2,whichModel))
    return (1/myAlpha)*esC

def integrateAllT(y,v,l,n,p,c,p1,p2,whichModel):
    pY = getPy(p,y,p1,p2,whichModel,v)
    d = getYDensity(y,p1,p2,whichModel,v)
    t_l = getSaddlePoint(pY,c,l)
    varPart = getVaRPart(l,n,pY,c,t_l)
    esPart = getESPart(l,n,pY,c,t_l)
    correctPart = getCorrectionPart(l,n,pY,c,t_l)
    return (varPart + esPart + correctPart)*d

def findAlphaGaussian(a,N,M,p,c,l,myRho):
    elTemp,ulTemp,varTemp,esTemp = th.oneFactorThresholdModel(N,M,p,c,
                         myRho,0,np.array([a]),0)
    return 1e4*(l-esTemp[0])**2

In [ ]:
import numpy as np
import cmUtilities as util
import importlib 
from scipy.stats import norm
from scipy.stats import t as myT
import scipy

import thresholdModels as th
import varContributions as vc

importlib.reload(th)
importlib.reload(vc)
importlib.reload(util)

def myFunction(x):
    return 7*np.sin(5*x) + 5*np.power(x,3) + 3*np.exp(x/2)

def mcIntegration(M):
    U = np.random.uniform(0,1,M)
    return np.mean(myFunction(U))

def naiveNumericalIntegration(K,a,b):
    myGrid = np.linspace(a,b,K)
    myIntegral = 0
    for n in range(0,K-1):
        dx = myGrid[n+1]-myGrid[n]
        myIntegral += myFunction(myGrid[n+1])*dx
    return myIntegral    

def getQ(theta,c,p):
    return np.divide(p*np.exp(c*theta),vc.computeMGF(theta,p,c))
    
def meanShiftOF(mu,c,p,l,myRho):
    pZ = th.computeP(p,myRho,mu)
    theta = vc.getSaddlePoint(pZ,c,l,0.0)
    f_l = -theta*l + vc.computeCGF(theta,pZ,c)
    return -(f_l - 0.5*np.dot(mu,mu))    

def getOptimalMeanShift(c,p,l,myRho):
    r = scipy.optimize.minimize(meanShiftOF,-1.0,args=(c,p,l,myRho), 
                            method='SLSQP',jac=None,bounds=[(-4.0,4.0)]) 
    return r.x

def isIndDefault(N,M,p,c,l):
    U = np.random.uniform(0,1,[M,N])
    theta = vc.getSaddlePoint(p,c,l,0.0)
    qZ = getQ(theta,c,p)
    cgf = vc.computeCGF(theta,p,c)
    I = np.transpose(1*np.less(U,qZ))
    L = np.dot(c,I)
    rn = computeRND(theta,L,cgf)        
    tailProb = np.mean(np.multiply(L>l,rn)) 
    eShortfall =  np.mean(np.multiply(L*(L>l),rn))/tailProb        
    return tailProb,eShortfall    

def isThreshold(N,M,p,c,l,myRho,nu,shiftMean,isT,invVector=0):
    mu = 0.0
    gamma = 0.0
    if shiftMean==1:
        mu = getOptimalMeanShift(c,p,l,myRho)
    theta = np.zeros(M)
    cgf = np.zeros(M)
    qZ = np.zeros([M,N])
    G = np.transpose(np.tile(np.random.normal(mu,1,M),(N,1)))
    e = np.random.normal(0,1,[M,N])
    if isT==1:
        gamma = -2
        W = np.random.chisquare(nu,M)
        myV = W/(1-2*gamma)
        V = np.transpose(np.sqrt(np.tile(myV,(N,1))/nu))
        num = (1/V)*myT.ppf(p,nu)*np.ones((M,1))-np.multiply(np.sqrt(myRho),G)
        pZ = norm.cdf(np.divide(num,np.sqrt(1-myRho)))
    elif isT==2:
        V = np.transpose(np.sqrt(np.tile(np.random.gamma(nu,1/nu,M),(N,1))))
        num = (1/V)*invVector*np.ones((M,1))-np.multiply(np.sqrt(myRho),G)
        pZ = norm.cdf(np.divide(num,np.sqrt(1-myRho)))
    else:
        pZ = th.computeP(p,myRho,G)
    for n in range(0,M):
        theta[n] = vc.getSaddlePoint(pZ[n,:],c,l,0.0)
        qZ[n,:] = getQ(theta[n],c,pZ[n,:])
        cgf[n] = vc.computeCGF(theta[n],pZ[n,:],c)
    I = np.transpose(1*np.less(e,norm.ppf(qZ)))
    L = np.dot(c,I)
    if isT==1:
        rnChi = np.exp(-gamma*myV-(nu/2)*np.log(1-2*gamma))
    else:
        rnChi = np.ones(M)
    if shiftMean==1:
        rn = computeRND(theta,L,cgf)*np.exp(-mu*G[:,0]+0.5*(mu**2))*rnChi
    else:
        rn = computeRND(theta,L,cgf)*rnChi
    tailProb = np.mean(np.multiply(L>l,rn)) 
    eShortfall =  np.mean(np.multiply(L*(L>l),rn))/tailProb        
    return tailProb,eShortfall    
              
def isMixture(N,M,p,c,l,p1,p2):
    theta = np.zeros(M)
    cgf = np.zeros(M)
    qS = np.zeros([M,N])
    S = np.random.gamma(p1, 1/p1, [M]) 
    wS =  np.transpose(np.tile(1-p2 + p2*S,[N,1]))
    pS = np.tile(p,[M,1])*wS
    for n in range(0,M):
        theta[n] = vc.getSaddlePoint(pS[n,:],c,l,-0.2)
        qS[n,:] = getQ(theta[n],c,pS[n,:])
        cgf[n] = vc.computeCGF(theta[n],pS[n,:],c)
    I = np.transpose(1*np.greater_equal(np.random.poisson(qS,[M,N]),1))
    L = np.dot(c,I)
    rn = computeRND(theta,L,cgf)        
    tailProb = np.mean(np.multiply(L>l,rn)) 
    eShortfall =  np.mean(np.multiply(L*(L>l),rn))/tailProb        
    return tailProb,eShortfall    

def computeRND(theta,L,cgf):
    return np.exp(-np.multiply(theta,L)+cgf)

def isThresholdSimple(N,M,p,c,l,myRho):
    mu = getOptimalMeanShift(c,p,l,myRho)
    theta = np.zeros(M)
    cgf = np.zeros(M)
    qZ = np.zeros([M,N])
    e = np.random.normal(0,1,[M,N])
    G = np.transpose(np.tile(np.random.normal(mu,1,M),(N,1)))
    num = (norm.ppf(p)*np.ones((M,1)))-np.sqrt(myRho)*G
    pZ = norm.cdf(np.divide(num,np.sqrt(1-myRho)))
    for n in range(0,M):
        theta[n] = vc.getSaddlePoint(pZ[n,:],c,l,0.0)
        qZ[n,:] = getQ(theta[n],c,pZ[n,:])
        cgf[n] = vc.computeCGF(theta[n],pZ[n,:],c)
    I = np.transpose(1*np.less(e,norm.ppf(qZ)))
    L = np.dot(c,I)
    rn = np.exp(-mu*G[:,0]+0.5*(mu**2))*computeRND(theta,L,cgf)
    tailProb = np.mean(np.multiply(L>l,rn)) 
    eShortfall =  np.mean(np.multiply(L*(L>l),rn))/tailProb        
    return tailProb,eShortfall    

def isThresholdT(N,M,p,c,l,myRho,nu,cm=0):
    myShift = (1-2*cm)
    mu = getOptimalMeanShift(c,p,l,myRho)
    W = np.random.chisquare(nu,M)
    myV = W/myShift
    theta = np.zeros(M)
    cgf = np.zeros(M)
    qZ = np.zeros([M,N])
    V = np.transpose(np.sqrt(np.tile(myV,(N,1))/nu))
    e = np.random.normal(0,1,[M,N])
    G = np.transpose(np.tile(np.random.normal(mu,1,M),(N,1)))
    num = V*(myT.ppf(p,nu)*np.ones((M,1)))-np.sqrt(myRho)*G
    pZ = norm.cdf(np.divide(num,np.sqrt(1-myRho)))
    for n in range(0,M):
        theta[n] = vc.getSaddlePoint(pZ[n,:],c,l,0.0)
        qZ[n,:] = getQ(theta[n],c,pZ[n,:])
        cgf[n] = vc.computeCGF(theta[n],pZ[n,:],c)
    I = np.transpose(1*np.less(e,norm.ppf(qZ)))
    L = np.dot(c,I)
    rnChi = np.exp(-cm*myV-(nu/2)*np.log(myShift))
    rnMu=np.exp(-mu*G[:,0]+0.5*(mu**2))
    rnTwist = computeRND(theta,L,cgf)
    rn = rnChi*rnMu*rnTwist
    tailProb = np.mean(np.multiply(L>l,rn)) 
    eShortfall =  np.mean(np.multiply(L*(L>l),rn))/tailProb        
    return tailProb,eShortfall    

def isThresholdContr(N,M,p,c,l,myRho,nu,cm=0):
    myShift = (1-2*cm)
    xhat = scipy.optimize.minimize(meanShiftOF,0.01, 
                                args=(c,p,l,myRho), 
                                method='SLSQP', jac=None)                             
    mu = xhat.x    
    theta = np.zeros(M)
    cgf = np.zeros(M)
    qZ = np.zeros([M,N])
    G = np.transpose(np.tile(np.random.normal(mu,1,M),(N,1)))
    e = np.random.normal(0,1,[M,N])
    W = np.random.chisquare(nu,M)
    myV = W/myShift
    V = np.transpose(np.sqrt(np.tile(myV,(N,1))/nu))
    num = V*myT.ppf(p,nu)*np.ones((M,1))-np.multiply(np.sqrt(myRho),G)
    pZ = norm.cdf(np.divide(num,np.sqrt(1-myRho)))
    for n in range(0,M):
        theta[n] = vc.getSaddlePoint(pZ[n,:],c,l,0.0)
        qZ[n,:] = getQ(theta[n],c,pZ[n,:])
        cgf[n] = vc.computeCGF(theta[n],pZ[n,:],c)
    I = np.transpose(1*np.less(e,norm.ppf(qZ)))
    L = np.dot(c,I)         
    rnChi=np.exp(-cm*myV-(nu/2)*np.log(myShift))
    rnMu=np.exp(-mu*G[:,0]+0.5*mu**2)
    rnTwist = computeRND(theta,L,cgf)
    rn = rnChi*rnMu*rnTwist
    return I,theta,pZ,qZ,cgf,rn

In [1]:
from __future__ import print_function
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Read Training dataset as well as drop the index column
training_data = pd.read_csv('./data/cs-training.csv').drop('Unnamed: 0', axis = 1)


# For each column heading we replace "-" and convert the heading in lowercase 
cleancolumn = []
for i in range(len(training_data.columns)):
    cleancolumn.append(training_data.columns[i].replace('-', '').lower())
training_data.columns = cleancolumn

# print the 5 records of the traiing dataset
training_data.head()

# Describe the all statistical properties of the training dataset
training_data[training_data.columns[1:]].describe()

training_data[training_data.columns[1:]].median()

training_data[training_data.columns[1:]].mean()

# This give you the calulation of the target lebels. Which category of the target lebel is how many percentage.
total_len = len(training_data['seriousdlqin2yrs'])
percentage_labels = (training_data['seriousdlqin2yrs'].value_counts()/total_len)*100
percentage_labels

# Graphical representation of the target label percentage.
sns.set()
sns.countplot(training_data.seriousdlqin2yrs).set_title('Data Distribution')
ax = plt.gca()
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2.,
            height + 2,
            '{:.2f}%'.format(100*(height/total_len)),
            fontsize=14, ha='center', va='bottom')
sns.set(font_scale=1.5)
ax.set_xlabel("Labels for seriousdlqin2yrs attribute")
ax.set_ylabel("Numbers of records")
plt.show()

# You will get to know which column has missing value and it's give the count that how many records are missing 
training_data.isnull().sum()

# Graphical representation of the missing values.
x = training_data.columns
y = training_data.isnull().sum()
sns.set()
sns.barplot(x,y)
ax = plt.gca()
for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2.,
            height + 2,
            int(height),
            fontsize=14, ha='center', va='bottom')
sns.set(font_scale=1.5)
ax.set_xlabel("Data Attributes")
ax.set_ylabel("count of missing records for each attribute")
plt.xticks(rotation=90)
plt.show()

# Actual replacement of the missing value using mean value.
training_data_mean_replace = training_data.fillna((training_data.mean()))
training_data_mean_replace.head()


training_data_mean_replace.isnull().sum()

# Actual replacement of the missing value using median value.
training_data_median_replace = training_data.fillna((training_data.median()))
training_data_median_replace.head()

training_data_median_replace.isnull().sum()

training_data.fillna((training_data.median()), inplace=True)
# Get the correlation of the training dataset
training_data[training_data.columns[1:]].corr()

sns.set()
sns.set(font_scale=1.25)
sns.heatmap(training_data[training_data.columns[1:]].corr(),annot=True,fmt=".1f")
plt.show()


# Percentile based outlier detection
def percentile_based_outlier(data, threshold=95):
    diff = (100 - threshold) / 2.0
    (minval, maxval) = np.percentile(data, [diff, 100 - diff])
    #return minval, maxval
    return ((data < minval) | (data > maxval))
#percentile_based_outlier(data=training_data.revolvingutilizationofunsecuredlines)

# Another percentile based outlier detection method which is based on inter quertile(IQR) range
# import numpy as np
# def outliers_iqr(ys):
#     quartile_1, quartile_3 = np.percentile(ys, [25, 75])
#     iqr = quartile_3 - quartile_1
#     lower_bound = quartile_1 - (iqr * 1.5)
#     upper_bound = quartile_3 + (iqr * 1.5)
#     return np.where((ys > upper_bound) | (ys < lower_bound))

def mad_based_outlier(points, threshold=3.5):
    median_y = np.median(points)
    median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in points])
    modified_z_scores = [0.6745 * (y - median_y) / median_absolute_deviation_y
                         for y in points]

    return np.abs(modified_z_scores) > threshold
#mad_based_outlier(points=training_data.age)


def std_div(data, threshold=3):
    std = data.std()
    mean = data.mean()
    isOutlier = []
    for val in data:
        if val/std > threshold:
            isOutlier.append(True)
        else:
            isOutlier.append(False)
    return isOutlier
#std_div(data=training_data.age)

def outlierVote(data):
    x = percentile_based_outlier(data)
    y = mad_based_outlier(data)
    z = std_div(data)
    temp = zip(data.index, x, y, z)
    final = []
    for i in range(len(temp)):
        if temp[i].count(False) >= 2:
            final.append(False)
        else:
            final.append(True)
    return final
#outlierVote(data=training_data.age)

def plotOutlier(x):
    fig, axes = plt.subplots(nrows=4)
    for ax, func in zip(axes, [percentile_based_outlier, mad_based_outlier, std_div, outlierVote]):
        sns.distplot(x, ax=ax, rug=True, hist=False)
        outliers = x[func(x)]
        ax.plot(outliers, np.zeros_like(outliers), 'ro', clip_on=False)

    kwargs = dict(y=0.95, x=0.05, ha='left', va='top', size=20)
    axes[0].set_title('Percentile-based Outliers', **kwargs)
    axes[1].set_title('MAD-based Outliers', **kwargs)
    axes[2].set_title('STD-based Outliers', **kwargs)
    axes[3].set_title('Majority vote based Outliers', **kwargs)
    fig.suptitle('Comparing Outlier Tests with n={}'.format(len(x)), size=20)
    fig = plt.gcf()
    fig.set_size_inches(15,10)
    
plotOutlier(training_data.revolvingutilizationofunsecuredlines.sample(5000))
plotOutlier(training_data.age.sample(1000))
plotOutlier(training_data.numberoftime3059dayspastduenotworse.sample(1000))
plotOutlier(training_data.debtratio.sample(1000))
plotOutlier(training_data.monthlyincome.sample(1000))
plotOutlier(training_data.numberofopencreditlinesandloans.sample(1000))
plotOutlier(training_data.numberoftimes90dayslate.sample(1000))
plotOutlier(training_data.numberrealestateloansorlines.sample(1000))
plotOutlier(training_data.numberoftime6089dayspastduenotworse.sample(1000))
plotOutlier(training_data.numberofdependents.sample(1000))

revNew = []
training_data.revolvingutilizationofunsecuredlines
for val in training_data.revolvingutilizationofunsecuredlines:
    if val <= 0.99999:
        revNew.append(val)
    else:
        revNew.append(0.99999)
training_data.revolvingutilizationofunsecuredlines = revNew
training_data.age.plot.box()

import collections
collections.Counter(training_data.age)

ageNew = []
for val in training_data.age:
    if val > 21:
        ageNew.append(val)
    else:
        ageNew.append(21)
        
training_data.age = ageNew

collections.Counter(training_data.numberoftime3059dayspastduenotworse)

New = []
med = training_data.numberoftime3059dayspastduenotworse.median()
for val in training_data.numberoftime3059dayspastduenotworse:
    if ((val == 98) | (val == 96)):
        New.append(med)
    else:
        New.append(val)

training_data.numberoftime3059dayspastduenotworse = New

def outlierRatio(data):
    functions = [percentile_based_outlier, mad_based_outlier, std_div, outlierVote]
    outlierDict = {}
    for func in functions:
        funcResult = func(data)
        count = 0
        for val in funcResult:
            if val == True:
                count += 1 
        outlierDict[str(func)[10:].split()[0]] = [count, '{:.2f}%'.format((float(count)/len(data))*100)]
    
    return outlierDict
outlierRatio(training_data.debtratio)

plotOutlier(training_data.debtratio.sample(1000))


def add_freq():
    ncount = len(training_data)

    ax2=ax.twinx()

    ax2.yaxis.tick_left()
    ax.yaxis.tick_right()

    ax.yaxis.set_label_position('right')
    ax2.yaxis.set_label_position('left')

    ax2.set_ylabel('Frequency [%]')

    for p in ax.patches:
        x=p.get_bbox().get_points()[:,0]
        y=p.get_bbox().get_points()[1,1]
        ax.annotate('{:.1f}%'.format(100.*y/ncount), (x.mean(), y), 
                ha='center', va='bottom')

    ax2.set_ylim(0,100)
    ax2.grid(None)
ax = sns.countplot(mad_based_outlier(training_data.debtratio))

add_freq()

minUpperBound = min([val for (val, out) in zip(training_data.debtratio, mad_based_outlier(training_data.debtratio)) if out == True])

newDebtRatio = []
for val in training_data.debtratio:
    if val > minUpperBound:
        newDebtRatio.append(minUpperBound)
    else:
        newDebtRatio.append(val)

training_data.debtratio = newDebtRatio 

def plotOutlierFree(x):
    fig, axes = plt.subplots(nrows=4)
    nOutliers = []
    for ax, func in zip(axes, [percentile_based_outlier, mad_based_outlier, std_div, outlierVote]):
        tfOutlier = zip(x, func(x))
        nOutliers.append(len([index for (index, bol) in tfOutlier if bol == True]))
        outlierFree = [index for (index, bol) in tfOutlier if bol == True]
        sns.distplot(outlierFree, ax=ax, rug=True, hist=False)
        
    kwargs = dict(y=0.95, x=0.05, ha='left', va='top', size=15)
    axes[0].set_title('Percentile-based Outliers, removed: {r}'.format(r=nOutliers[0]), **kwargs)
    axes[1].set_title('MAD-based Outliers, removed: {r}'.format(r=nOutliers[1]), **kwargs)
    axes[2].set_title('STD-based Outliers, removed: {r}'.format(r=nOutliers[2]), **kwargs)
    axes[3].set_title('Majority vote based Outliers, removed: {r}'.format(r=nOutliers[3]), **kwargs)
    fig.suptitle('Outlier Removed By Method with n={}'.format(len(x)), size=20)
    fig = plt.gcf()
    fig.set_size_inches(15,10)
    
plotOutlierFree(training_data.monthlyincome.sample(1000)) 
    
def replaceOutlier(data, method = outlierVote, replace='median'):
    '''replace: median (auto)
                'minUpper' which is the upper bound of the outlier detection'''
    vote = outlierVote(data)
    x = pd.DataFrame(zip(data, vote), columns=['debt', 'outlier'])
    if replace == 'median':
        replace = x.debt.median()
    elif replace == 'minUpper':
        replace = min([val for (val, vote) in zip(data, vote) if vote == True])
        if replace < data.mean():
            return 'There are outliers lower than the sample mean'
    debtNew = []
    for i in range(x.shape[0]):
        if x.iloc[i][1] == True:
            debtNew.append(replace)
        else:
            debtNew.append(x.iloc[i][0])
    
    return debtNew
    
    
incomeNew = replaceOutlier(training_data.monthlyincome, replace='minUpper')

training_data.monthlyincome = incomeNew

collections.Counter(training_data.numberoftimes90dayslate)

def removeSpecificAndPutMedian(data, first = 98, second = 96):
    New = []
    med = data.median()
    for val in data:
        if ((val == first) | (val == second)):
            New.append(med)
        else:
            New.append(val)
            
    return New

new = removeSpecificAndPutMedian(training_data.numberoftimes90dayslate)

training_data.numberoftimes90dayslate = new

collections.Counter(training_data.numberrealestateloansorlines)

realNew = []
for val in training_data.numberrealestateloansorlines:
    if val > 17:
        realNew.append(17)
    else:
        realNew.append(val) 
training_data.numberrealestateloansorlines = realNew

collections.Counter(training_data.numberoftime6089dayspastduenotworse)

new = removeSpecificAndPutMedian(training_data.numberoftime6089dayspastduenotworse)
training_data.numberoftime6089dayspastduenotworse = new

collections.Counter(training_data.numberofdependents)

depNew = []
for var in training_data.numberofdependents:
    if var > 10:
        depNew.append(10)
    else:
        depNew.append(var)

training_data.numberofdependents = depNew

training_data.head()

from sklearn.ensemble import RandomForestClassifier

training_data.columns[1:]

X = training_data.drop('seriousdlqin2yrs', axis=1)
y = training_data.seriousdlqin2yrs
features_label = training_data.columns[1:]
forest = RandomForestClassifier (n_estimators = 10000, random_state=0, n_jobs = -1)
forest.fit(X,y)
importances = forest.feature_importances_
indices = np. argsort(importances)[::-1]
for i in range(X.shape[1]):
    print ("%2d) %-*s %f" % (i + 1, 30, features_label[i],importances[indices[i]]))

plt.title('Feature Importances')
plt.bar(range(X.shape[1]),importances[indices], color="green", align="center")
plt.xticks(range(X.shape[1]),features_label, rotation=90)
plt.xlim([-1, X.shape[1]])
plt.show()


from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score

X = training_data.drop('seriousdlqin2yrs', axis=1)
y = training_data.seriousdlqin2yrs

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

knMod = KNeighborsClassifier(n_neighbors=5, weights='uniform', algorithm='auto', leaf_size=30, p=2, metric='minkowski', metric_params=None)

knMod.fit(X_train, y_train)

knMod.score(X_test, y_test)

test_labels=knMod.predict_proba(np.array(X_test.values))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

glmMod = LogisticRegression(penalty='l1', dual=False, tol=0.0001, C=1.0, fit_intercept=True,
                            intercept_scaling=1, class_weight=None, 
                            random_state=None, solver='liblinear', max_iter=100,
                            multi_class='ovr', verbose=2)


glmMod.fit(X_train, y_train)

glmMod.score(X_test, y_test)

test_labels=glmMod.predict_proba(np.array(X_test.values))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

adaMod = AdaBoostClassifier(base_estimator=None, n_estimators=200, learning_rate=1.0)

adaMod.fit(X_train, y_train)

adaMod.score(X_test, y_test)

test_labels=adaMod.predict_proba(np.array(X_test.values))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

gbMod = GradientBoostingClassifier(loss='deviance', learning_rate=0.1, n_estimators=200, subsample=1.0,
                                   min_samples_split=2, min_samples_leaf=1, min_weight_fraction_leaf=0.0, 
                                   max_depth=3,
                                   init=None, random_state=None, max_features=None, verbose=0)

gbMod.fit(X_train, y_train)

gbMod.score(X_test, y_test)

test_labels=gbMod.predict_proba(np.array(X_test.values))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

rfMod = RandomForestClassifier(n_estimators=10, criterion='gini', max_depth=None, min_samples_split=2,
                               min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto',
                               max_leaf_nodes=None, bootstrap=True, oob_score=False, n_jobs=1, 
                               random_state=None, verbose=0)
rfMod.fit(X_train, y_train)

rfMod.score(X_test, y_test)

test_labels=rfMod.predict_proba(np.array(X_test.values))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

from sklearn.model_selection import cross_val_score
def cvDictGen(functions, scr, X_train=X, y_train=y, cv=5, verbose=1):
    cvDict = {}
    for func in functions:
        cvScore = cross_val_score(func, X_train, y_train, cv=cv, verbose=verbose, scoring=scr)
        cvDict[str(func).split('(')[0]] = [cvScore.mean(), cvScore.std()]
    
    return cvDict

def cvDictNormalize(cvDict):
    cvDictNormalized = {}
    for key in cvDict.keys():
        for i in cvDict[key]:
            cvDictNormalized[key] = ['{:0.2f}'.format((cvDict[key][0]/cvDict[cvDict.keys()[0]][0])),
                                     '{:0.2f}'.format((cvDict[key][1]/cvDict[cvDict.keys()[0]][1]))]
    return cvDictNormalized

cvD = cvDictGen(functions=[knMod, glmMod, adaMod, gbMod, rfMod], scr='roc_auc')
cvD

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

adaHyperParams = {'n_estimators': [10,50,100,200,420]}

gridSearchAda = RandomizedSearchCV(estimator=adaMod, param_distributions=adaHyperParams, n_iter=5,
                                   scoring='roc_auc', fit_params=None, cv=None, verbose=2).fit(X_train, y_train)

gridSearchAda.best_params_, gridSearchAda.best_score_

gbHyperParams = {'loss' : ['deviance', 'exponential'],
                 'n_estimators': randint(10, 500),
                 'max_depth': randint(1,10)}

gridSearchGB = RandomizedSearchCV(estimator=gbMod, param_distributions=gbHyperParams, n_iter=10,
                                   scoring='roc_auc', fit_params=None, cv=None, verbose=2).fit(X_train, y_train)

gridSearchGB.best_params_, gridSearchGB.best_score_


bestGbModFitted = gridSearchGB.best_estimator_.fit(X_train, y_train)

bestAdaModFitted = gridSearchAda.best_estimator_.fit(X_train, y_train)

cvDictbestpara = cvDictGen(functions=[bestGbModFitted, bestAdaModFitted], scr='roc_auc')

cvDictbestpara

test_labels=bestGbModFitted.predict_proba(np.array(X_test.values))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

test_labels=bestAdaModFitted.predict_proba(np.array(X_test.values))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p)
X_train_1 = np.array(X_train)
X_train_transform = transformer.transform(X_train_1)

bestGbModFitted_transformed = gridSearchGB.best_estimator_.fit(X_train_transform, y_train)

bestAdaModFitted_transformed = gridSearchAda.best_estimator_.fit(X_train_transform, y_train)

cvDictbestpara_transform = cvDictGen(functions=[bestGbModFitted_transformed, bestAdaModFitted_transformed],
                                     scr='roc_auc')

cvDictbestpara_transform

import numpy as np
from sklearn.preprocessing import FunctionTransformer

transformer = FunctionTransformer(np.log1p)
X_test_1 = np.array(X_test)
X_test_transform = transformer.transform(X_test_1)

X_test_transform

test_labels=bestGbModFitted_transformed.predict_proba(np.array(X_test_transform))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

test_labels=bestAdaModFitted_transformed.predict_proba(np.array(X_test_transform))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)


from sklearn.ensemble import VotingClassifier
votingMod = VotingClassifier(estimators=[('gb', bestGbModFitted_transformed), 
                                         ('ada', bestAdaModFitted_transformed)], voting='soft',weights=[2,1])
votingMod = votingMod.fit(X_train_transform, y_train)


test_labels=votingMod.predict_proba(np.array(X_test_transform))[:,1]

votingMod.score(X_test_transform, y_test)

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)

from sklearn.ensemble import VotingClassifier
votingMod_old = VotingClassifier(estimators=[('gb', bestGbModFitted), ('ada', bestAdaModFitted)], 
                                 voting='soft',weights=[2,1])
votingMod_old = votingMod.fit(X_train, y_train)

test_labels=votingMod_old.predict_proba(np.array(X_test.values))[:,1]

roc_auc_score(y_test,test_labels , average='macro', sample_weight=None)




FileNotFoundError: [Errno 2] No such file or directory: './data/cs-training.csv'

In [27]:
import yfinance as yf

msft = yf.Ticker("AAPL")
#msft.info
msft.history_metadata

stock = yf.Ticker(ticker)                           
temp = pd.DataFrame(stock.dividends)
div = temp.loc[start:end]

msft.get_shares_full(start="2022-01-01", end=None)

msft.income_stmt
msft.quarterly_income_stmt

msft.balance_sheet
msft.quarterly_balance_sheet

msft.cashflow
msft.quarterly_cashflow

# show holders
msft.major_holders
msft.institutional_holders
msft.mutualfund_holders

# show news
msft.news

msft.options #option dates
opt = msft.option_chain('2025-12-19')
